# 관광지 데이터 전처리

1. 비짓제주 크롤링 데이터로 필터링
2. 관광지 입장 데이터로 가격정보 추가
3. 데이터 병합
4. 네이버 맵 크롤링 데이터로 분류정보 보완
5. 탄소배출량 데이터 병합
6. 데이터 정리 
7. 주소 오류 보안


In [1]:
import pandas as pd
import numpy as np
import re
import os
import math
import json
import requests

import warnings
warnings.filterwarnings(action='ignore')

## 1. 비짓제주 크롤링 데이터로 필터링


In [2]:
def name_preprocessing(name):
    name = name.replace(' (', '(').replace(') ', ')').strip()
    name = name.replace('(주)','').replace("·",".").lower()
    name = re.sub('[\(\)㈜!@#$%^&*{}|:;\'\"/><,~`+=_ ]','', name)
    name = re.sub('[\[.\]]','',name)
    return name.strip()

In [3]:
#주소 형식 맞추기
def addr_preprocessing(addr):
    addr = re.sub('제주 서귀포시','제주특별자치도 서귀포시',addr)
    addr = re.sub('제주 제주시','제주특별자치도 제주시',addr)

    while (addr.split()[-1][-1].isdigit() == False) : #상세주소 제거
        if len(addr.split()) <= 3 : break
            
        addr = " ".join(addr.split()[0:-1]) #하나빼기

    addr = "".join(addr.split()) #공백다 없애기
    return addr.strip() #앞뒤정리

In [4]:
#주소 형식 맞추기
def addr_preprocessing2(addr):
    addr = re.sub('제주 서귀포시','제주특별자치도 서귀포시',addr)
    addr = re.sub('제주 제주시','제주특별자치도 제주시',addr)

    while addr.split()[-1][-1].isdigit() == False : #상세주소 제거
        if addr.split()[-1][-1] == "리" : break
            
        addr = " ".join(addr.split()[0:-1])

    addr = "".join(addr.split())
    return addr.strip()

In [5]:
#주소 형식 맞추기
def addr_preprocessing3(addr):

    while addr.split()[-1][-1].isdigit() == False : #상세주소 제거
        if len(addr.split()) <= 3 : break
            
        addr = " ".join(addr.split()[0:-1]) #하나빼기

    addr = "".join(addr.split()) #공백다 없애기
    return addr.strip() #앞뒤정리

In [6]:
vj_view = pd.read_csv('data/PRE_FINAL_JT_MT_ACCTO_TRRSRT_SCCNT_LIST.csv', encoding='utf-8-sig')
vj_craw_spot = pd.read_csv('data/VISITJEJU_관광지정보크롤링.csv')

In [7]:
# 원래 있던 기본 데이터에서 관광지만 남김
vj_view_spot = vj_view[vj_view['CL_NM']=='관광지']
vj_view_spot = vj_view_spot.reset_index(drop=True) 

#주소 전처리
vj_view_spot['ADDR_PRE'] = vj_view_spot['ADDR'].str.replace(pat="['",repl="",regex=False)
vj_view_spot['ADDR_PRE'] = vj_view_spot['ADDR_PRE'].str.replace(pat="']",repl="",regex=False)

vj_view_spot['ADDR_PRE'] = list(map(lambda x : addr_preprocessing(x), vj_view_spot['ADDR_PRE']))
vj_view_spot.head()

,AREA_NM_PRE,CL_NM,AREA_NM,ADDR,BASE_YEAR,MAR_VIEW_CO,APR_VIEW_CO,MAY_VIEW_CO,SEP_VIEW_CO,OCT_VIEW_CO,NOV_VIEW_CO,ADDR_PRE
0,1100고지습지,관광지,['1100고지습지'],['제주특별자치도 서귀포시 색달동 산 1-2'],12129,613,561,608,494,624,960,제주특별자치도서귀포시색달동산1-2
1,1100고지휴게소,관광지,['1100고지(휴게소)'],['제주특별자치도 서귀포시 1100로 1555'],12129,438,420,506,323,451,541,제주특별자치도서귀포시1100로1555
2,1112도로,관광지,['1112도로'],['제주특별자치도 제주시 명림로 584 (봉개동)'],12129,60,53,53,109,119,112,제주특별자치도제주시명림로584
3,43해원방사탑,관광지,['4·3해원방사탑'],['제주특별자치도 제주시 신산로 82'],12129,34,45,53,45,56,47,제주특별자치도제주시신산로82
4,516도로숲터널,관광지,['5.16도로숲터널'],['제주특별자치도 서귀포시 남원읍 신례리 산 2-1'],12129,179,154,173,161,171,156,제주특별자치도서귀포시남원읍신례리산2-1


In [8]:
# 크롤링 데이터 이름 preprocessing -> 새로운 열 만듦
vj_craw_spot['name_PRE'] = list(map(lambda x : name_preprocessing(x), vj_craw_spot['name']))

In [9]:
# 안 쓸 열들 drop
vj_craw_spot = vj_craw_spot.drop(['stars', 'address', 'telephone', 'numreview', 'view', 'rec_tour', 'rec_restaurant', 'rec_lodgment', 'rec_bigdata'], axis=1) 

In [10]:
#가능한 많은 데이터를 merge하기 위해 outer 활용
checker2 = pd.merge(left = vj_view_spot, right = vj_craw_spot, how = 'outer', left_on = 'AREA_NM_PRE', right_on = 'name_PRE').reset_index(drop=True)
checker2.tail(20)

,AREA_NM_PRE,CL_NM,AREA_NM,ADDR,BASE_YEAR,MAR_VIEW_CO,APR_VIEW_CO,MAY_VIEW_CO,SEP_VIEW_CO,OCT_VIEW_CO,NOV_VIEW_CO,ADDR_PRE,name,tags,like,heart,detail_text,detail_side_box,name_PRE
1322,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,제주베스트힐 펜션&글램핑,#야영장 #캠핑장 #숙소 #휴양펜션 #펜션 #독채 #한옥 #온돌방 #바비큐 #수영장...,1.0,1,제주 베스트힐 펜션은 제주의 허파라 불리는 곶자왈 숲 속 바농오름 자락에 자리잡고 ...,"{'소개': '글램핑이 있는 자연 속 팬션', '이용 시간': 'Checkin : ...",제주베스트힐펜션글램핑
1323,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,제주샘주 영농협동조합,#술 #오메기술 #고소리술 #칵테일만들기 체험 #토속주 #니모메술 #세우리술 #음식...,0.0,7,제주샘주 영농협동조합 제주도 서쪽에는 제주를 대표하는 오메기떡과 감귤을 술로 만나...,"{'소개': '제주를 대표하는 오메기떡과 감귤을 술로 만나볼 수 있는 제주샘주', ...",제주샘주영농협동조합
1324,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,제주시 충혼묘지 4·3추모비,#4.3 #문화유적지 #문화관광 #제주4.3 #역사유적 #어트랙션 #제주4.3 #역...,0.0,1,제주시 충혼묘지에는 제주도민에게 잘 알려진 박진경 대령 추도비가 세워져 있다. 그는...,{'소개': '제주시 충혼묘지에 세워진 박진경 대령 추도비.'},제주시충혼묘지43추모비
1325,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,제주에코승마장,#부모 #커플 #혼자 #친구 #아이 #맑음 #액티비티 #승마 #체험 #레저/체험 #...,0.0,24,"트레킹은 초보자를 대상으로 운영하는 세 가지 코스와 단체, 기업인을 대상으로 하는 ...","{'소개': '진정한 제주를 경험하길 원하신다면 제주에코승마파크 친구, 연인, 가족...",제주에코승마장
1326,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,제주잠수함,#액티비티 #부모 #아이 #신흥 #체험 #레저/체험 #어린이 #수상레저 #어트랙션 ...,1.0,20,에메랄드빛 제주 바다에서 즐기는 해양레포츠 테마파크 국제리더스클럽! 국제리더스클럽...,{'소개': '에메랄드빛 제주 바다에서 즐기는 해양레포츠 테마파크 국제리더스클럽!'...,제주잠수함
1327,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,제주파인비치카라반,#야영장 #캠핑장 #숙소 #캠핑 #바비큐 #공공와이파이존 #온돌방 #언택트 #캠핑 ...,0.0,1,"야자수와 잔디정원, 바다가 어우러져 이국적인 제주를 느낄 수 있는 캠핑장. 올레길...","{'소개': '야자수와 잔디정원, 바다가 어우러져 이국적인 제주를 느낄 수 있는 캠...",제주파인비치카라반
1328,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,제주펫,#애완동물 #카페 #음식,1.0,0,'반려인과 반려동물이 함께 행복을 꿈꾸는 곳' 제주펫은 반려동물과 함께 놀 수 있...,"{'소개': '반려인과 반려동물이 함께 행복을 꿈꾸는 곳', '상세 정보': '영업...",제주펫
1329,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,제주평화박물관,#실내 #미술/박물관 #아이 #비.눈 #가을 #실내관광지 #문화관광 #박물관 #어트...,2.0,24,현재 제주평화박물관은 휴관중입니다. 방문전 전화 등으로 영업재개 여부를 확인하시고 ...,"{'소개': ""일제강점기 '과거의 이야기'를 직접 체험하면서 평화의 소중함을 느낄 ...",제주평화박물관
1330,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,주정공장,#4.3 #문화유적지 #문화관광 #제주4.3 #역사유적 #어트랙션 #제주4.3 #역...,0.0,0,1934년 일제에 의해 설립된 동양척식주식회사(약칭 동척회사) 제주주정공장은 해방전...,{'소개': '4.3사건 당시 수용소로 활용한 동양척식주식회사 제주주정공장.'},주정공장
1331,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,중문색달해변,#부모 #커플 #혼자 #친구 #아이 #경관/포토 #휴식/힐링 #사계절 #청년 #중/...,1.0,40,중문색달해변 중문색달해변은 서퍼들의 천국이라 불릴 만큼 파도가 높아 여름에는 다이...,{'소개': '서퍼들의 천국이라 불릴 만큼 파도가 높아 여름에는 다이나믹한 해수욕을...,중문색달해변


In [11]:
check_list=checker2[checker2.AREA_NM_PRE.isna()].name_PRE
name_list = []
idx=[]
for i in range(0,len(check_list)):
    ck = checker2[checker2.AREA_NM_PRE.isna()].name_PRE.iloc[i]
    check_df = checker2[(checker2['AREA_NM_PRE'].str.contains(ck,na=False)) & (checker2['name_PRE'].isna())]
    check_df = pd.concat([check_df,checker2[checker2['name_PRE'] == ck]], ignore_index=False)
    
    if len(check_df) == 2 :
        name_list.append(ck)
        idx.append(check_df.index)
print(len(name_list),len(idx))

101 101


In [12]:
checker2.iloc[idx[0]]

,AREA_NM_PRE,CL_NM,AREA_NM,ADDR,BASE_YEAR,MAR_VIEW_CO,APR_VIEW_CO,MAY_VIEW_CO,SEP_VIEW_CO,OCT_VIEW_CO,NOV_VIEW_CO,ADDR_PRE,name,tags,like,heart,detail_text,detail_side_box,name_PRE
546,성산일출봉unesco세계자연유산,관광지,['성산일출봉(UNESCO 세계자연유산)'],['제주특별자치도 서귀포시 성산읍 일출로 284-12'],12129.0,6022.0,9274.0,12332.0,5916.0,7946.0,7643.0,제주특별자치도서귀포시성산읍일출로284-12,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1211,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,성산일출봉,#일출 #오름 #경관/포토 #부모 #자연경관 #포토스팟 #유네스코,281.0,"3,570",현재 성산일출봉은 새로운 일상을 위하여 사회적 거리두기 의무화 조치가 해제되었습니다...,"{'소개': '바다위에 우뚝 솟아난 수성화산·유네스코 세계자연유산, 천연기념물 제4...",성산일출봉


101개가 이름 전처리 문제 등으로 합쳐지지 않은 것이 의심됨

In [13]:
def adj_rows(main_idx, sub_idx):
    checker2.at[main_idx, 'name'] = checker2.at[sub_idx, 'name']
    checker2.at[main_idx, 'tags'] = checker2.at[sub_idx, 'tags']
    checker2.at[main_idx, 'like'] = checker2.at[sub_idx, 'like']
    checker2.at[main_idx, 'heart'] = checker2.at[sub_idx, 'heart']
    checker2.at[main_idx, 'name_PRE'] = checker2.at[sub_idx, 'name_PRE']

In [14]:
for k in range(0,len(idx)):
    adj_rows(idx[k][0], idx[k][1])

In [15]:
checker2.iloc[idx[0]] #수정된 내용 확인

,AREA_NM_PRE,CL_NM,AREA_NM,ADDR,BASE_YEAR,MAR_VIEW_CO,APR_VIEW_CO,MAY_VIEW_CO,SEP_VIEW_CO,OCT_VIEW_CO,NOV_VIEW_CO,ADDR_PRE,name,tags,like,heart,detail_text,detail_side_box,name_PRE
546,성산일출봉unesco세계자연유산,관광지,['성산일출봉(UNESCO 세계자연유산)'],['제주특별자치도 서귀포시 성산읍 일출로 284-12'],12129.0,6022.0,9274.0,12332.0,5916.0,7946.0,7643.0,제주특별자치도서귀포시성산읍일출로284-12,성산일출봉,#일출 #오름 #경관/포토 #부모 #자연경관 #포토스팟 #유네스코,281.0,"3,570",NaN,NaN,성산일출봉
1211,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,성산일출봉,#일출 #오름 #경관/포토 #부모 #자연경관 #포토스팟 #유네스코,281.0,"3,570",현재 성산일출봉은 새로운 일상을 위하여 사회적 거리두기 의무화 조치가 해제되었습니다...,"{'소개': '바다위에 우뚝 솟아난 수성화산·유네스코 세계자연유산, 천연기념물 제4...",성산일출봉


In [16]:
checker2 = checker2.dropna(subset = ['AREA_NM_PRE', 'name_PRE']).reset_index(drop=True)
checker2 = checker2.drop(['CL_NM','AREA_NM','ADDR','BASE_YEAR','name'],axis = 1)
checker2

,AREA_NM_PRE,MAR_VIEW_CO,APR_VIEW_CO,MAY_VIEW_CO,SEP_VIEW_CO,OCT_VIEW_CO,NOV_VIEW_CO,ADDR_PRE,tags,like,heart,detail_text,detail_side_box,name_PRE
0,1100고지습지,613.0,561.0,608.0,494.0,624.0,960.0,제주특별자치도서귀포시색달동산1-2,#산 #경관/포토 #흐림 #가을 #자연경관 #포토스팟 #한라산 #언택트 #가을 #자...,3.0,110,1100고지 습지는 한라산 고원지대에 형성된 대표적인 산지습지로서 16개 이상의 습...,{'소개': '쉽게 만나볼 수 없는 야생동물을 만나볼 수 있는 곳'},1100고지습지
1,1100고지휴게소,438.0,420.0,506.0,323.0,451.0,541.0,제주특별자치도서귀포시1100로1555,#산 #경관/포토 #흐림 #겨울 #포토스팟 #한라산 #언택트 #어트랙션 #제주명품4...,7.0,284,NaN,NaN,1100고지
2,1112도로,60.0,53.0,53.0,109.0,119.0,112.0,제주특별자치도제주시명림로584,#산 #드라이브 #흐림 #자연경관 #자연경관,4.0,125,‘대한민국에서 가장 아름다운 도로’ 중 하나로 길 양 옆으로 하늘로 곧게 뻗은 삼나...,"{'소개': ""‘대한민국에서 가장 아름다운 도로' 중 하나로 길 양 옆으로 하늘로 ...",1112도로
3,43해원방사탑,34.0,45.0,53.0,45.0,56.0,47.0,제주특별자치도제주시신산로82,#4.3 #문화유적지 #문화관광 #제주4.3 #역사유적 #어트랙션 #제주4.3 #역...,0.0,1,"4․3 해원방사탑(解寃防邪塔)은 1998년 4월 3일, 4․3 50주년을 맞아 「제...",{'소개': '다시는 이 땅에 4․3과 같은 비극이 재발되지 않도록 돌을 쌓아 만든...,43해원방사탑
4,516도로숲터널,179.0,154.0,173.0,161.0,171.0,156.0,제주특별자치도서귀포시남원읍신례리산2-1,#드라이브 #맑음 #자연경관 #숲 #단풍 #제주명품42 #포토스팟 #제주7대도로,3.0,190,5.16 도로를 지나다 보면 도로 양쪽의 나무들이 아치형으로 도로를 감싸고 있는 듯...,{'소개': '제주시와 서귀포를 잇는 낭만적인 숲터널'},516도로숲터널
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1066,후포해변,44.0,52.0,43.0,57.0,42.0,48.0,제주특별자치도제주시추자면대서5길87,#해변 #경관/포토 #자연경관 #포토스팟 #포토스팟,0.0,4,파도에 부딪혀 동글동글해진 몽돌들이 있는 해변이다.,{'소개': '탈의실과 편의시설을 갖춘 작은 휴양지'},후포해변
1067,훈데르트바서파크,219.0,720.0,251.0,0.0,0.0,0.0,제주특별자치도제주시우도면우도해안길32-12,#문화관광 #건축문화기행 #테마공원 #포토스팟 #포토스팟,0.0,9,"천혜 자연의 섬, 제주도 우도가 예술의 섬으로 탈바꿈했다. 세계적인 건축가 겸 환경...","{'소개': ""오스트리아 3대 화가이자 환경 운동가, 건축 치료사 ‘훈데르트바서' ...",훈데르트바서파크
1068,휘닉스르쏠레이테라피,79.0,81.0,142.0,112.0,122.0,122.0,제주특별자치도서귀포시성산읍섭지코지로107,#스파 #체험 #호텔 #어트랙션 #어트랙션,7.0,5,"스파, 커플룸, 프라이빗룸으로 구성된 르쏠레이테라피센터는 성산 휘닉스호텔앤리조트 내...","{'소개': '스파, 커플룸, 프라이빗룸으로 구성된 르쏠레이테라피센터는 성산 휘닉스...",휘닉스르쏠레이테라피
1069,휴림,273.0,248.0,254.0,1205.0,1396.0,825.0,제주특별자치도제주시애월읍광령남서길40,#부모 #친구 #아이 #커플 #액티비티 #야영장 #캠핑장 #체험 #레저/체험 #어린...,0.0,8,숲속쉼터 휴림은 제주시 산림조합이 조성하여 2017년 개장한 에코힐링파크로 남녀노소...,"{'소개': '제주시 산림조합이 조성한 숲속 쉼터, 에코힐링파크', '편의시설': ...",휴림


In [17]:
checker2['AREA_NM_PRE'].value_counts()

좌보미오름         2
라온주실내체험동물원    2
지그리오름         2
제주올레10-1코스    1
플레이박스vr       1
             ..
케이제주씨워킹       1
휘닉스르쏠레이테라피    1
발자국화석공원       1
곤흘동곤을동마을터     1
송계순집터         1
Name: AREA_NM_PRE, Length: 1068, dtype: int64

In [18]:
checker2[(checker2['AREA_NM_PRE'] == '지그리오름')+(checker2['AREA_NM_PRE'] == '좌보미오름')+(checker2['AREA_NM_PRE'] == '라온주실내체험동물원')]

,AREA_NM_PRE,MAR_VIEW_CO,APR_VIEW_CO,MAY_VIEW_CO,SEP_VIEW_CO,OCT_VIEW_CO,NOV_VIEW_CO,ADDR_PRE,tags,like,heart,detail_text,detail_side_box,name_PRE
226,라온주실내체험동물원,63.0,91.0,120.0,84.0,97.0,98.0,제주특별자치도제주시한림읍월림7길155,#아이 #테마공원 #실내관광지 #체험 #어린이 #어트랙션 #어린이동물농장 #체험 #...,3.0,11,"더마파크 안에 자리한 라온주(ZOO)는 포유류와 파충류, 조류, 어류에 이르기까지 ...","{'소개': '더마파크 안에 자리한 라온주(ZOO)는 포유류와 파충류, 조류, 어류...",라온주실내체험동물원
227,라온주실내체험동물원,63.0,91.0,120.0,84.0,97.0,98.0,제주특별자치도제주시한림읍월림7길155,#아이 #테마공원 #실내관광지 #체험 #어린이 #어트랙션 #어린이동물농장,3.0,11,"더마파크 안에 자리한 라온주(ZOO)는 포유류와 파충류, 조류, 어류에 이르기까지 ...","{'소개': '더마파크 안에 자리한 라온주(ZOO)는 포유류와 파충류, 조류, 어류...",라온주실내체험동물원
882,좌보미오름,47.0,55.0,40.0,50.0,57.0,47.0,제주특별자치도서귀포시표선면성읍리산6,#커플 #친구 #혼자 #맑음 #오름 #자연경관 #도보여행 #도보 #맑음 #오름 #자...,0.0,8,"오름은 주소가 산지로 되어있어, 내비게이션 검색시 정확한 위치를 알 수 없거나, 주...",{'소개': '제주오름의 투박한 아름다움을 그대로 간직한 오름'},좌보미오름
883,좌보미오름,47.0,55.0,40.0,50.0,57.0,47.0,제주특별자치도서귀포시표선면성읍리산6,#커플 #친구 #혼자 #맑음 #오름 #자연경관 #도보여행 #도보,0.0,8,"오름은 주소가 산지로 되어있어, 내비게이션 검색시 정확한 위치를 알 수 없거나, 주...",{'소개': '제주오름의 투박한 아름다움을 그대로 간직한 오름'},좌보미오름
897,지그리오름,30.0,38.0,40.0,38.0,26.0,38.0,제주특별자치도제주시조천읍교래리산137-1,#오름 #걷기/등산 #자연경관 #도보여행 #도보 #도보여행 #도보,0.0,4,"오름은 주소가 산지로 되어있어, 내비게이션 검색시 정확한 위치를 알 수 없거나, 주...",{'소개': '울창한 숲과 곶자왈 지대로 이루어진 지그리오름'},지그리오름
898,지그리오름,30.0,38.0,40.0,38.0,26.0,38.0,제주특별자치도제주시조천읍교래리산137-1,#오름 #걷기/등산 #자연경관 #도보여행 #도보 #도보여행 #도보,0.0,4,"오름은 주소가 산지로 되어있어, 내비게이션 검색시 정확한 위치를 알 수 없거나, 주...",{'소개': '아담한 풀밭사면에 자연림으로 이루어진 오름'},지그리오름


In [19]:
checker2 = checker2.drop([227,883,898], axis=0).reset_index(drop=True) 

In [20]:
#checker2.to_csv('PRE_1_관광지데이터1011.csv', encoding='utf-8-sig', index=False)

In [21]:
vj_final=checker2

## 2. 관광지 입장 데이터 전처리
관광지 입장 데이터(JT_TRRSRT_ENTRN_LIST) 전처리 및 입장객수, 분류값, 가격정보 추가

In [22]:
enter = pd.read_csv('data/JT_TRRSRT_ENTRN_LIST.csv',encoding='CP949')
enter_spot = enter[(enter.CL_NM != "음식점") | (enter.CL_NM != "숙박") ]

In [23]:
enter_spot.isna().sum()

CL_NM                 0
TRRSRT_NM             0
ADDR                  0
TEL_NO               93
TOT_ENTRN_NMPR_CO     0
SETLE_PRICE           0
ENTRN_DE              0
dtype: int64

In [24]:
enter_spot['ENTRN_MN'] = enter_spot['ENTRN_DE'].astype(str).str[4:6]

# 안 쓸 열들 drop
enter_spot = enter_spot.drop(['TEL_NO', 'ENTRN_DE'],axis=1)

In [25]:
# 입장 데이터 이름 전처리-> 새로운 열 만듦
enter_spot['TRRSRT_NM_PRE'] = list(map(lambda x : name_preprocessing(x), enter_spot['TRRSRT_NM'])) ; enter_spot

,CL_NM,TRRSRT_NM,ADDR,TOT_ENTRN_NMPR_CO,SETLE_PRICE,ENTRN_MN,TRRSRT_NM_PRE
0,기타,봄그리고가을리조트,제주 서귀포시 성산읍 해맞이해안로 2660 시흥근생및주택,2,18000,03,봄그리고가을리조트
1,기타,토스카나호텔,제주 서귀포시 용흥로66번길 158-7,2,57230,03,토스카나호텔
2,기타,산방산탄산온천,제주 서귀포시 안덕면 사계북로41번길 192,5,37500,03,산방산탄산온천
3,기타,해비치호텔앤드리조트(섬모라),제주 서귀포시 표선면 민속해안로 537,1,30600,03,해비치호텔앤드리조트섬모라
4,레저관광,우도잠수함,제주 서귀포시 성산읍 성산등용로 112-7,2,82400,03,우도잠수함
...,...,...,...,...,...,...,...
1048569,공연전시,빛의벙커,제주 서귀포시 성산읍 고성리 2039-22,2,36000,05,빛의벙커
1048570,공연전시,빛의벙커,제주 서귀포시 성산읍 고성리 2039-22,1,18000,05,빛의벙커
1048571,테마파크,스누피가든,제주 제주시 구좌읍 금백조로 930,2,36000,05,스누피가든
1048572,공연전시,점보빌리지,제주 서귀포시 안덕면 평화로319번길 31-11,3,32500,05,점보빌리지


In [26]:
# 입장 데이터 주소 전처리-> 새로운 열 만듦
enter_spot['ADDR_PRE'] = list(map(lambda x : addr_preprocessing2(x), enter_spot['ADDR'])) ; enter_spot

,CL_NM,TRRSRT_NM,ADDR,TOT_ENTRN_NMPR_CO,SETLE_PRICE,ENTRN_MN,TRRSRT_NM_PRE,ADDR_PRE
0,기타,봄그리고가을리조트,제주 서귀포시 성산읍 해맞이해안로 2660 시흥근생및주택,2,18000,03,봄그리고가을리조트,제주특별자치도서귀포시성산읍해맞이해안로2660
1,기타,토스카나호텔,제주 서귀포시 용흥로66번길 158-7,2,57230,03,토스카나호텔,제주특별자치도서귀포시용흥로66번길158-7
2,기타,산방산탄산온천,제주 서귀포시 안덕면 사계북로41번길 192,5,37500,03,산방산탄산온천,제주특별자치도서귀포시안덕면사계북로41번길192
3,기타,해비치호텔앤드리조트(섬모라),제주 서귀포시 표선면 민속해안로 537,1,30600,03,해비치호텔앤드리조트섬모라,제주특별자치도서귀포시표선면민속해안로537
4,레저관광,우도잠수함,제주 서귀포시 성산읍 성산등용로 112-7,2,82400,03,우도잠수함,제주특별자치도서귀포시성산읍성산등용로112-7
...,...,...,...,...,...,...,...,...
1048569,공연전시,빛의벙커,제주 서귀포시 성산읍 고성리 2039-22,2,36000,05,빛의벙커,제주특별자치도서귀포시성산읍고성리2039-22
1048570,공연전시,빛의벙커,제주 서귀포시 성산읍 고성리 2039-22,1,18000,05,빛의벙커,제주특별자치도서귀포시성산읍고성리2039-22
1048571,테마파크,스누피가든,제주 제주시 구좌읍 금백조로 930,2,36000,05,스누피가든,제주특별자치도제주시구좌읍금백조로930
1048572,공연전시,점보빌리지,제주 서귀포시 안덕면 평화로319번길 31-11,3,32500,05,점보빌리지,제주특별자치도서귀포시안덕면평화로319번길31-11


In [27]:
# 특이값 전처리
enter_spot['TRRSRT_NM_PRE']=enter_spot['TRRSRT_NM_PRE'].str.replace(pat='생각하는정원분재예술원',repl='생각하는정원',regex=False)
enter_spot['TRRSRT_NM_PRE']=enter_spot['TRRSRT_NM_PRE'].str.replace(pat='마라도정기여객선운진항',repl='마라도정기여객선',regex=False)

index = enter_spot[enter_spot['ADDR_PRE'] == '제주시한경면노을해안로1163'].index
enter_spot['ADDR_PRE'].iloc[index] = '제주특별자치도제주시한경면노을해안로1163'

In [28]:
#일별 데이터 -> 월별 데이터로 처리하기 
#전처리되지 않은 정보는 빼고 처리
enter_grouped = enter_spot.groupby(['CL_NM', 'TRRSRT_NM_PRE', 'ADDR_PRE','ENTRN_MN']).sum()
enter_grouped = enter_grouped.reset_index()
enter_grouped

,CL_NM,TRRSRT_NM_PRE,ADDR_PRE,ENTRN_MN,TOT_ENTRN_NMPR_CO,SETLE_PRICE
0,공연전시,건강과성박물관,제주특별자치도서귀포시안덕면일주서로1611,03,340,3497770
1,공연전시,건강과성박물관,제주특별자치도서귀포시안덕면일주서로1611,04,322,3546600
2,공연전시,건강과성박물관,제주특별자치도서귀포시안덕면일주서로1611,05,233,2385300
3,공연전시,고흐의정원,제주특별자치도서귀포시성산읍삼달신풍로126-5,03,38,446700
4,공연전시,고흐의정원,제주특별자치도서귀포시성산읍삼달신풍로126-5,04,86,552100
...,...,...,...,...,...,...
415,테마파크,허브동산,제주특별자치도서귀포시표선면돈오름로170,04,211,2012100
416,테마파크,허브동산,제주특별자치도서귀포시표선면돈오름로170,05,232,1940870
417,테마파크,화조원,제주특별자치도제주시애월읍애원로804,03,628,8583320
418,테마파크,화조원,제주특별자치도제주시애월읍애원로804,04,986,13797150


In [29]:
#상호명과 주소는 같은데 분류가 다른 경우, 최빈값으로 수정
check3 = enter_grouped[['TRRSRT_NM_PRE','ADDR_PRE','CL_NM']].groupby(['TRRSRT_NM_PRE','ADDR_PRE']).nunique()
check3 = check3[check3.CL_NM>1]

for i in range(0,len(check3)):
    chck_index=enter_grouped[(enter_grouped.TRRSRT_NM_PRE == check3.index[i][0]) * (enter_grouped.ADDR_PRE == check3.index[i][1])].index
    chck_list= enter_grouped.CL_NM[chck_index]
    cat_name = chck_list[chck_list!="기타"].mode()
    
    enter_grouped.CL_NM[chck_index] = cat_name[0]

In [30]:
# 가격,입장수 관련 열로 만들기
enter_price = enter_grouped.pivot(index=['CL_NM','TRRSRT_NM_PRE', 'ADDR_PRE'], columns='ENTRN_MN', values='SETLE_PRICE')
enter_price.columns = ['MAR_VISIT_PRICE', 'APR_VISIT_PRICE','MAY_VISIT_PRICE','SEP_VISIT_PRICE','OCT_VISIT_PRICE','NOV_VISIT_PRICE']
enter_price = enter_price.reset_index()


enter_num = enter_grouped.pivot(index=['CL_NM', 'TRRSRT_NM_PRE', 'ADDR_PRE'], columns='ENTRN_MN', values='TOT_ENTRN_NMPR_CO')
enter_num.columns = ['MAR_VISIT_NUM', 'APR_VISIT_NUM','MAY_VISIT_ NUM','SEP_VISIT_NUM','OCT_VISIT_NUM','NOV_VISIT_NUM']
enter_num = enter_num.reset_index()
enter_num

,CL_NM,TRRSRT_NM_PRE,ADDR_PRE,MAR_VISIT_NUM,APR_VISIT_NUM,MAY_VISIT_ NUM,SEP_VISIT_NUM,OCT_VISIT_NUM,NOV_VISIT_NUM
0,공연전시,건강과성박물관,제주특별자치도서귀포시안덕면일주서로1611,340.0,322.0,233.0,NaN,NaN,386.0
1,공연전시,고흐의정원,제주특별자치도서귀포시성산읍삼달신풍로126-5,38.0,86.0,121.0,NaN,NaN,NaN
2,공연전시,그대나의뮤즈,제주특별자치도서귀포시안덕면신화역사로304번길89,265.0,60.0,NaN,NaN,NaN,586.0
3,공연전시,그림포레스트,제주특별자치도서귀포시중문관광로110번길32,NaN,NaN,199.0,NaN,NaN,NaN
4,공연전시,노형수퍼마켙,제주특별자치도제주시노형로89,586.0,418.0,403.0,NaN,NaN,585.0
...,...,...,...,...,...,...,...,...,...
116,테마파크,쇠소깍산물관광농원,제주특별자치도서귀포시남원읍하례로90,9.0,NaN,4.0,NaN,NaN,2.0
117,테마파크,제주돌마을공원,제주특별자치도제주시한림읍금능남로421,2.0,14.0,11.0,NaN,12.0,7.0
118,테마파크,제주무민랜드,제주특별자치도서귀포시안덕면병악로420,477.0,588.0,363.0,208.0,834.0,NaN
119,테마파크,카멜리아힐,제주특별자치도서귀포시안덕면병악로166,10631.0,14205.0,21916.0,NaN,NaN,21928.0


In [31]:
#최종 입장 데이터
enter_final = pd.merge(left = enter_price, right = enter_num, how = 'outer', on = ['CL_NM','TRRSRT_NM_PRE','ADDR_PRE']).reset_index(drop=True)
enter_final

,CL_NM,TRRSRT_NM_PRE,ADDR_PRE,MAR_VISIT_PRICE,APR_VISIT_PRICE,MAY_VISIT_PRICE,SEP_VISIT_PRICE,OCT_VISIT_PRICE,NOV_VISIT_PRICE,MAR_VISIT_NUM,APR_VISIT_NUM,MAY_VISIT_ NUM,SEP_VISIT_NUM,OCT_VISIT_NUM,NOV_VISIT_NUM
0,공연전시,건강과성박물관,제주특별자치도서귀포시안덕면일주서로1611,3497770.0,3546600.0,2385300.0,NaN,NaN,3653630.0,340.0,322.0,233.0,NaN,NaN,386.0
1,공연전시,고흐의정원,제주특별자치도서귀포시성산읍삼달신풍로126-5,446700.0,552100.0,782700.0,NaN,NaN,NaN,38.0,86.0,121.0,NaN,NaN,NaN
2,공연전시,그대나의뮤즈,제주특별자치도서귀포시안덕면신화역사로304번길89,3916000.0,885000.0,NaN,NaN,NaN,8562550.0,265.0,60.0,NaN,NaN,NaN,586.0
3,공연전시,그림포레스트,제주특별자치도서귀포시중문관광로110번길32,NaN,NaN,2005000.0,NaN,NaN,NaN,NaN,NaN,199.0,NaN,NaN,NaN
4,공연전시,노형수퍼마켙,제주특별자치도제주시노형로89,8180400.0,5862550.0,5824610.0,NaN,NaN,8938020.0,586.0,418.0,403.0,NaN,NaN,585.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
116,테마파크,쇠소깍산물관광농원,제주특별자치도서귀포시남원읍하례로90,203200.0,NaN,69400.0,NaN,NaN,8000.0,9.0,NaN,4.0,NaN,NaN,2.0
117,테마파크,제주돌마을공원,제주특별자치도제주시한림읍금능남로421,0.0,49000.0,57500.0,NaN,66000.0,32500.0,2.0,14.0,11.0,NaN,12.0,7.0
118,테마파크,제주무민랜드,제주특별자치도서귀포시안덕면병악로420,5189300.0,5066250.0,3319890.0,3155300.0,13138680.0,NaN,477.0,588.0,363.0,208.0,834.0,NaN
119,테마파크,카멜리아힐,제주특별자치도서귀포시안덕면병악로166,81497040.0,113663400.0,169525260.0,NaN,NaN,171953570.0,10631.0,14205.0,21916.0,NaN,NaN,21928.0


In [32]:
enter_final = enter_final[(enter_final.CL_NM != "음식점") * (enter_final.CL_NM != "숙박") ]

In [33]:
#enter_final.to_csv('PRE_2_관광지데이터1011.csv', encoding='utf-8-sig', index=False)

## 3.  데이터 병합

In [34]:
#vj_final = pd.read_csv('PRE_1_관광지데이터1011.csv',encoding='utf-8-sig')
#enter_final = pd.read_csv('PRE_2_관광지데이터1011.csv',encoding='utf-8-sig')

In [35]:
# 제공 데이터 이름 기준으로 left join
final_1 = pd.merge(left = vj_final, right = enter_final, how = 'left', left_on = ['AREA_NM_PRE'],right_on = ['TRRSRT_NM_PRE']).reset_index(drop=True)

place_df= final_1[final_1.CL_NM.isna()==False] #merge 성공
idx = final_1[final_1.CL_NM.isna()].index #merge 안된애들 index

# 크롤링데이터 이름 기준으로 left join
final_2 = pd.merge(left = vj_final.iloc[idx], right = enter_final, how = 'left', left_on = ['name_PRE'],right_on = ['TRRSRT_NM_PRE']).reset_index(drop=True)

# merge된 데이터 합치기

place_df = place_df.append(final_2)
place_df = place_df.reset_index(drop=True)
place_df.shape

(1068, 29)

In [36]:
place_df.CL_NM.isna().sum() #입장데이터에서 가져온 것은 53

1015

In [37]:
#주소 확인
place_df[place_df['ADDR_PRE_x'] != place_df['ADDR_PRE_y']][['AREA_NM_PRE','TRRSRT_NM_PRE','ADDR_PRE_x','ADDR_PRE_y']]

,AREA_NM_PRE,TRRSRT_NM_PRE,ADDR_PRE_x,ADDR_PRE_y
4,꽃썸,꽃썸,서귀포시안덕면동광로151,제주특별자치도서귀포시안덕면동광로151
24,용머리해안,용머리해안,제주특별자치도서귀포시안덕면,제주특별자치도서귀포시안덕면사계리
33,제주실탄사격장,제주실탄사격장,제주특별자치도서귀포시소보리당로,제주특별자치도서귀포시소보리당로164번길62
35,제주카약,제주카약,제주특별자치도제주시조천읍평사길19,제주특별자치도제주시구좌읍해맞이해안로1950
38,천제연폭포,천제연폭포,제주특별자치도서귀포시천제연로,제주특별자치도서귀포시천제연로132
...,...,...,...,...
1063,후포해변,NaN,제주특별자치도제주시추자면대서5길87,NaN
1064,훈데르트바서파크,NaN,제주특별자치도제주시우도면우도해안길32-12,NaN
1065,휘닉스르쏠레이테라피,NaN,제주특별자치도서귀포시성산읍섭지코지로107,NaN
1066,휴림,NaN,제주특별자치도제주시애월읍광령남서길40,NaN


In [38]:
place_df['ADDR_PRE'] = np.NaN

#입장 데이터에서 가져온 주소가 있으면 그것을 기준으로 한다
idx_y = place_df[place_df['ADDR_PRE_y'].isna()==False].index
place_df.ADDR_PRE.iloc[idx_y] = place_df.ADDR_PRE_y.iloc[idx_y]

#입장 데이터가 없는 경우, 비짓제주데이터를 기준으로 한다
idx_x = place_df[place_df['ADDR_PRE_y'].isna()].index
place_df.ADDR_PRE.iloc[idx_x] = place_df.ADDR_PRE_x.iloc[idx_x]

#글자 "nan"이 불필요하게 들어있는 경우와, 주소가 두 개씩 입력되어 있는 경우 처리
place_df['ADDR_PRE']=place_df['ADDR_PRE'].str.replace(pat='nan',repl='',regex=False)
place_df.ADDR_PRE[416] = "제주특별자치도제주시한경면청수리6"
place_df.ADDR_PRE[622] = "제주특별자치도제주시한경면고락로"
place_df.ADDR_PRE[927] = "제주특별자치도서귀포시서귀동961-33"
place_df.ADDR_PRE[56] = "제주특별자치도서귀포시대정읍최남단해안로128"
place_df.ADDR_PRE[1050] = "제주특별자치도제주시화북일동4008"

In [39]:
#카카오api로 지번주소, 위도, 경도를 가져온다
api_key = '7465f66e54fbe4f71543e474623685f1' #숨김처리

In [40]:
place_df['LTNO_ADDR'] = np.NaN
place_df['GPS_LNGTD'] = np.NaN
place_df['GPS_LTTD'] = np.NaN

addrs = place_df.ADDR_PRE

for i in range(0,len(addrs)) :
    try : 
        url = 'https://dapi.kakao.com/v2/local/search/address.json?query={}'.format(addrs[i])
        headers = {"Authorization": "KakaoAK " + api_key}
        result = json.loads(str(requests.get(url, headers = headers).text))
    
        place_df['LTNO_ADDR'].iloc[i] = result['documents'][0]['address']['address_name']
        place_df['GPS_LNGTD'].iloc[i] = result['documents'][0]['address']['x'] #경도
        place_df['GPS_LTTD'].iloc[i] = result['documents'][0]['address']['y'] #위도
    
    except: #카카오맵에서 검색 안됨
        place_df['LTNO_ADDR'].iloc[i] = np.NaN
        place_df['GPS_LNGTD'].iloc[i] = np.NaN
        place_df['GPS_LTTD'].iloc[i] = np.NaN

In [41]:
#주어진 데이터가 도로명주소가 아닌 지번주소인 경우(361개)가 있었지만 그대로 잘 들어갔다
no_addr_index = place_df[-(place_df['ADDR_PRE'].str.contains("로") + place_df['ADDR_PRE'].str.contains("길"))].index
place_df[['AREA_NM_PRE','name_PRE','ADDR_PRE','LTNO_ADDR','GPS_LNGTD']].iloc[no_addr_index]

,AREA_NM_PRE,name_PRE,ADDR_PRE,LTNO_ADDR,GPS_LNGTD
24,용머리해안,용머리해안,제주특별자치도서귀포시안덕면사계리,제주특별자치도 서귀포시 안덕면 사계리,126.306514122101
32,제주바다체험장,제주바다체험장,제주특별자치도제주시구좌읍동복리608-3,제주특별자치도 제주시 구좌읍 동복리 608-3,126.721836813919
49,1100고지습지,1100고지습지,제주특별자치도서귀포시색달동산1-2,제주특별자치도 서귀포시 색달동 산 1-2,126.462896523457
53,516도로숲터널,516도로숲터널,제주특별자치도서귀포시남원읍신례리산2-1,제주특별자치도 서귀포시 남원읍 신례리 산 2-1,126.605070169993
62,가메오름,가메오름,제주특별자치도제주시애월읍봉성리산125,제주특별자치도 제주시 애월읍 봉성리 산 125,126.335576565279
...,...,...,...,...,...
1051,화북환해장성,화북환해장성,제주특별자치도제주시화북동,제주특별자치도 제주시 화북동,126.565465645477
1054,화순금모래해수욕장담수수영장,화순금모래해수욕장담수수영장,제주특별자치도서귀포시안덕면화순리776-8,제주특별자치도 서귀포시 안덕면 화순리 776-8,126.335648464354
1055,화순리유채꽃길,화순리유채꽃길,제주특별자치도서귀포시안덕면화순리1230-4,제주특별자치도 서귀포시 안덕면 화순리 1230-4,126.333867094749
1060,황우지해안,황우지해안,제주특별자치도서귀포시서홍동766-1,제주특별자치도 서귀포시 서홍동 766-1,126.549766357796


In [42]:
# 전처리 오류로 주소가 정보가 충분하지 않은 경우 카카오API결과를 충분히 신뢰할 수 없으므로, POI데이터를 활용하여 보완한다.
no_number_index = place_df[-place_df['ADDR_PRE'].str[-1].str.isdigit()].index
place_df[['AREA_NM_PRE','ADDR_PRE','LTNO_ADDR','GPS_LNGTD']].iloc[no_number_index]

,AREA_NM_PRE,ADDR_PRE,LTNO_ADDR,GPS_LNGTD
24,용머리해안,제주특별자치도서귀포시안덕면사계리,제주특별자치도 서귀포시 안덕면 사계리,126.306514122101
66,가파도,제주특별자치도서귀포시대정읍,제주특별자치도 서귀포시 대정읍,126.251812381529
77,강정천유원지,제주특별자치도서귀포시강정통물로,NaN,NaN
79,개오리오름,제주특별자치도제주시봉개동,제주특별자치도 제주시 봉개동,126.594687244154
82,거믄여해안경승지,제주특별자치도서귀포시516로,NaN,NaN
...,...,...,...,...
1026,함덕카페거리,제주특별자치도제주시조천읍,제주특별자치도 제주시 조천읍,126.634132237491
1032,해맞이해안로,제주특별자치도제주시구좌읍,제주특별자치도 제주시 구좌읍,126.852053599141
1040,현사포구,제주특별자치도제주시이호1동,제주특별자치도 제주시 이호일동,126.457478118712
1044,협재포구,제주특별자치도제주시한림읍,제주특별자치도 제주시 한림읍,126.26694453422


In [43]:
# 전처리 오류로 한글에서 끊긴 경우 > poi 데이터로 보완하여 다시 API를 돌린다.
poi = pd.read_csv("data/제주특별자치도_제주도장소(POI)데이터_20151231.csv", encoding='CP949')

name_list = place_df['name_PRE'].iloc[no_number_index]
poi = poi[poi.장소명.isin(name_list)]
poi

,장소 POI에 대한 ID 구분 번호 체계,위치좌표 X축값,위치좌표 Y축값,구분,장소명,소재지,데이터기준일자
2,5,126.599675,33.230314,지명관련,섶섬,제주특별자치도 서귀포시 보목동 산 1,2015-12-31
3,6,126.565891,33.226031,지명관련,문섬,제주특별자치도 서귀포시 서귀동 산 4,2015-12-31
4,7,126.516395,33.218414,지명관련,범섬,제주특별자치도 서귀포시 법환동 산 2,2015-12-31
6,9,126.270455,33.170157,지명관련,가파도,제주특별자치도 서귀포시 대정읍 가파리 288,2015-12-31
63,67,126.551761,33.519340,지명관련,별도봉,제주특별자치도 제주시 화북일동 4461,2015-12-31
...,...,...,...,...,...,...,...
57689,58224,126.435677,33.238894,교통시설,대포포구,제주특별자치도 서귀포시 대포동 2415,2015-12-31
58329,58930,126.585038,33.299789,교통시설,돈내코,제주특별자치도 서귀포시 상효동 1460,2015-12-31
60436,62000,126.618757,33.428684,지명관련,개오리오름,제주특별자치도 제주시 봉개동 산 78-1,2015-12-31
60461,62025,126.350859,33.245525,지명관련,월라봉,제주특별자치도 서귀포시 안덕면 감산리 1148,2015-12-31


In [44]:
poi_name =  poi['장소명']
for i in range(0,len(poi)):
    idx = place_df[place_df['name_PRE']==poi_name.iloc[i]].index
    place_df['ADDR_PRE'].iloc[idx] = poi['소재지'].iloc[i]
    
    try : 
        url = 'https://dapi.kakao.com/v2/local/search/address.json?query={}'.format(poi['소재지'].iloc[i])
        headers = {"Authorization": "KakaoAK " + api_key}
        result = json.loads(str(requests.get(url, headers = headers).text))
    
        place_df['LTNO_ADDR'].iloc[idx] = result['documents'][0]['address']['address_name']
        place_df['GPS_LNGTD'].iloc[idx] = result['documents'][0]['address']['x'] #경도
        place_df['GPS_LTTD'].iloc[idx] = result['documents'][0]['address']['y'] #위도
    
    except: #카카오맵에서 검색 안됨
        pass

In [45]:
place_df[['AREA_NM_PRE','ADDR_PRE','LTNO_ADDR','GPS_LNGTD']].iloc[no_number_index]

,AREA_NM_PRE,ADDR_PRE,LTNO_ADDR,GPS_LNGTD
24,용머리해안,제주특별자치도 서귀포시 안덕면 사계리 181-1,제주특별자치도 서귀포시 안덕면 사계리 181-1,126.312581976897
66,가파도,제주특별자치도 서귀포시 대정읍 가파리 288,제주특별자치도 서귀포시 대정읍 가파리 288,126.270177260805
77,강정천유원지,제주특별자치도 서귀포시 강정동 2677,제주특별자치도 서귀포시 강정동 2677,126.489324445986
79,개오리오름,제주특별자치도 제주시 봉개동 산 78-1,제주특별자치도 제주시 봉개동 산 78-1,126.628940941105
82,거믄여해안경승지,제주특별자치도서귀포시516로,NaN,NaN
...,...,...,...,...
1026,함덕카페거리,제주특별자치도제주시조천읍,제주특별자치도 제주시 조천읍,126.634132237491
1032,해맞이해안로,제주특별자치도 서귀포시 성산읍 시흥리 3-1,제주특별자치도 서귀포시 성산읍 시흥리 3-1,126.910801060073
1040,현사포구,제주특별자치도제주시이호1동,제주특별자치도 제주시 이호일동,126.457478118712
1044,협재포구,제주특별자치도제주시한림읍,제주특별자치도 제주시 한림읍,126.26694453422


In [46]:
place_df.isna().sum() # 그럼에도 불구하고 주소 없음 또는 검색 안됨 등의 이유로 좌표 정보가 채워지지 않은 경우가 있다.

AREA_NM_PRE           0
MAR_VIEW_CO           0
APR_VIEW_CO           0
MAY_VIEW_CO           0
SEP_VIEW_CO           0
OCT_VIEW_CO           0
NOV_VIEW_CO           0
ADDR_PRE_x            0
tags                  0
like                  0
heart                 0
detail_text         102
detail_side_box     101
name_PRE              0
CL_NM              1015
TRRSRT_NM_PRE      1015
ADDR_PRE_y         1015
MAR_VISIT_PRICE    1031
APR_VISIT_PRICE    1029
MAY_VISIT_PRICE    1029
SEP_VISIT_PRICE    1038
OCT_VISIT_PRICE    1037
NOV_VISIT_PRICE    1023
MAR_VISIT_NUM      1031
APR_VISIT_NUM      1029
MAY_VISIT_ NUM     1029
SEP_VISIT_NUM      1038
OCT_VISIT_NUM      1037
NOV_VISIT_NUM      1023
ADDR_PRE              0
LTNO_ADDR            44
GPS_LNGTD            44
GPS_LTTD             44
dtype: int64

In [47]:
#보기 좋게 column순서를 정리하면서 drop한다
#이름을 기준으로 입장데이터를 붙였으니 더이상 TRRSRT_NM_PRE는 필요 없다

place_df = place_df[['CL_NM', 'AREA_NM_PRE',  'name_PRE', 'ADDR_PRE', 'LTNO_ADDR', 'GPS_LNGTD', 'GPS_LTTD', 
                     'tags', 'like', 'heart','detail_text','detail_side_box',
                     'MAR_VIEW_CO', 'APR_VIEW_CO', 'MAY_VIEW_CO', 'SEP_VIEW_CO', 'OCT_VIEW_CO', 'NOV_VIEW_CO',
                     'MAR_VISIT_PRICE','APR_VISIT_PRICE', 'MAY_VISIT_PRICE', 'SEP_VISIT_PRICE', 'OCT_VISIT_PRICE', 'NOV_VISIT_PRICE',
                     'MAR_VISIT_NUM', 'APR_VISIT_NUM', 'MAY_VISIT_ NUM', 'SEP_VISIT_NUM', 'OCT_VISIT_NUM', 'NOV_VISIT_NUM']]
place_df

,CL_NM,AREA_NM_PRE,name_PRE,ADDR_PRE,LTNO_ADDR,GPS_LNGTD,GPS_LTTD,tags,like,heart,...,MAY_VISIT_PRICE,SEP_VISIT_PRICE,OCT_VISIT_PRICE,NOV_VISIT_PRICE,MAR_VISIT_NUM,APR_VISIT_NUM,MAY_VISIT_ NUM,SEP_VISIT_NUM,OCT_VISIT_NUM,NOV_VISIT_NUM
0,테마관광지,감귤박물관,감귤박물관,제주특별자치도서귀포시효돈순환로441,제주특별자치도 서귀포시 신효동 1200-6,126.607428496419,33.271593041773,#실내 #미술/박물관 #아이 #비.눈 #겨울 #실내관광지 #문화관광 #박물관 #어트...,5.0,190,...,NaN,2317550.0,11208150.0,3987450.0,NaN,NaN,NaN,3587.0,16014.0,5550.0
1,공연전시,건강과성박물관,건강과성박물관,제주특별자치도서귀포시안덕면일주서로1611,제주특별자치도 서귀포시 안덕면 감산리 1736,126.346583683696,33.2548469661363,#실내 #미술/박물관 #커플 #비.눈 #사계절 #실내관광지 #문화관광 #박물관 #어...,6.0,110,...,2385300.0,NaN,NaN,3653630.0,340.0,322.0,233.0,NaN,NaN,386.0
2,공연전시,고흐의정원,고흐의정원,제주특별자치도서귀포시성산읍삼달신풍로126-5,제주특별자치도 서귀포시 성산읍 삼달리 1185-2,126.844086486445,33.3734094632983,#부모 #커플 #혼자 #친구 #아이 #사계절 #미술 #체험전시 #자연경관 #실내관광...,0.0,53,...,782700.0,NaN,NaN,NaN,38.0,86.0,121.0,NaN,NaN,NaN
3,레저관광,김녕요트투어,김녕요트투어,제주특별자치도제주시구좌읍구좌해안로229-16,제주특별자치도 제주시 구좌읍 김녕리 4212-1,126.7372597689,33.5582479162063,#액티비티 #커플 #여름 #체험 #레저/체험 #수상레저 #어트랙션 #체험 #레저/체...,3.0,144,...,3300000.0,NaN,NaN,2872000.0,24.0,59.0,72.0,NaN,NaN,62.0
4,기타,꽃썸,꽃썸,제주특별자치도서귀포시안덕면동광로151,제주특별자치도 서귀포시 안덕면 동광리 950-3,126.34015745353,33.3097350065685,#해변 #카페 #포토스팟 #휴식 #부모 #친구 #커플 #혼자 #안개/흐림 #맑음 #...,0.0,50,...,NaN,NaN,NaN,NaN,6.0,2.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1063,NaN,후포해변,후포해변,제주특별자치도제주시추자면대서5길87,제주특별자치도 제주시 추자면 대서리 267,126.28954186844,33.9623416994265,#해변 #경관/포토 #자연경관 #포토스팟 #포토스팟,0.0,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1064,NaN,훈데르트바서파크,훈데르트바서파크,제주특별자치도제주시우도면우도해안길32-12,제주특별자치도 제주시 우도면 연평리 1706-1,126.955738014169,33.4931582693186,#문화관광 #건축문화기행 #테마공원 #포토스팟 #포토스팟,0.0,9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1065,NaN,휘닉스르쏠레이테라피,휘닉스르쏠레이테라피,제주특별자치도서귀포시성산읍섭지코지로107,제주특별자치도 서귀포시 성산읍 고성리 127-2,126.926442688389,33.4260423785235,#스파 #체험 #호텔 #어트랙션 #어트랙션,7.0,5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1066,NaN,휴림,휴림,제주특별자치도제주시애월읍광령남서길40,제주특별자치도 제주시 애월읍 광령리 2873,126.428787959221,33.4437798639377,#부모 #친구 #아이 #커플 #액티비티 #야영장 #캠핑장 #체험 #레저/체험 #어린...,0.0,8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [48]:
#주소 미리 전처리
index = place_df[place_df['LTNO_ADDR'].isna()==False].index
place_df.LTNO_ADDR.iloc[index] =list(map(lambda x : addr_preprocessing3(x), place_df.iloc[index].LTNO_ADDR))

In [49]:
#place_df.to_csv('PRE_3_관광지데이터1011.csv', encoding='utf-8-sig', index=False)

## 4. 네이버 맵 크롤링 데이터로 정보 보완


In [50]:
#place_df = pd.read_csv('PRE_3_관광지데이터1003.csv',encoding='utf-8-sig')

### 4-(1) 네이버 크롤링 데이터 전처리

In [51]:
naver_craw = pd.read_csv('data/관광지데이터_네이버크롤링.csv', encoding='utf-8-sig') # 네이버지도 크롤링 자료
naver_craw

,0,1,2,3,4,5,6,7,8,9
0,서귀포감귤박물관,박물관,제주 서귀포시 효돈순환로 441,2180,883,0,NaN,NaN,NaN,NaN
1,건강과 성 박물관,박물관,제주 서귀포시 안덕면 일주서로 1611 건강과 성 박물관,4415,543,0,NaN,NaN,NaN,NaN
2,고흐의정원,"관람,체험",제주 서귀포시 성산읍 삼달신풍로 126-5,1361,370,0,NaN,NaN,NaN,NaN
3,김녕요트투어,요트,제주 제주시 구좌읍 구좌해안로 229-16,2011,489,0,NaN,NaN,NaN,NaN
4,꽃썸카페,"카페,디저트",제주 서귀포시 안덕면 동광로 151,27,13,0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
1063,후포해변,"해수욕장,해변",제주 제주시 추자면 대서리,6,8,NaN,NaN,NaN,NaN,NaN
1064,훈데르트바서파크,테마파크,제주 제주시 우도면 우도해안길 32-12,347,202,"{'볼거리가 많아요': 245, '사진이 잘 나와요': 169, '주차하기 편해요'...",NaN,NaN,NaN,NaN
1065,휘닉스르쏠레이테라피,no_result,no_result,no_result,0,0,no_result,no_result,no_result,no_result
1066,휴림,"캠핑,야영장",제주 제주시 애월읍 광령남서길 40,136,113,"{'그늘이 많아요': 33, '조용히 쉬기 좋아요': 29, '주차하기 편해요': ...",NaN,NaN,NaN,NaN


In [52]:
# column 이름 재설정
naver_craw.columns = ['nv_name', 'nv_type', 'nv_address', 'nv_review_num','nv_blog_num', 'nv_review', 'nv_dl_mood', 'nv_dl_topic', 'nv_dl_purpose', 'nv_dl_20']

#장소이름 전처리
naver_craw.nv_name =list(map(lambda x : name_preprocessing(x), naver_craw.nv_name))
naver_craw['nv_name'] = naver_craw['nv_name'].str.replace(pat="제주올레길",repl="제주올레",regex=False)


# 주소 정보 전처리
naver_craw['nv_address'] = naver_craw['nv_address'].str.replace(pat="제주특별자치 ",repl="제주특별자치도 ",regex=False)

index = naver_craw[(naver_craw['nv_address'].isna()==False) * (naver_craw['nv_address']!= "no_result")].index
naver_craw.nv_address.iloc[index] =list(map(lambda x : addr_preprocessing(x), naver_craw.iloc[index].nv_address))

In [53]:
#네이버지도에서 검색되지 않은 관광지 192개
naver_craw[naver_craw.nv_type=="no_result"].shape

(192, 10)

In [54]:
# 크롤링 데이터의 적절성 확인1 : 이름이 일치하는가
chk_index1 = []
for i in range(0,len(naver_craw)):
    chk_name = naver_craw.nv_name.iloc[i]
    
    if (place_df.AREA_NM_PRE.iloc[i] not in chk_name) * (place_df.name_PRE.iloc[i] not in chk_name)  * (chk_name not in place_df.name_PRE.iloc[i])  != 0 : #어디도 포함되지 않으면
        chk_index1.append(i)     

In [55]:
# 크롤링 데이터의 적절성 확인2 : 주소가 (동까지) 일치하는가
index = naver_craw[(naver_craw['nv_address'].isna()==False) * (naver_craw['nv_address']!= "no_result")].index
chk_df = place_df.iloc[index]
chk_nv = naver_craw.iloc[index]
chk_index2 = chk_df[(chk_df.ADDR_PRE.str[0:13]!=chk_nv.nv_address.str[0:13]) * (chk_df.LTNO_ADDR.str[0:13]!=chk_nv.nv_address.str[0:13])].index

In [56]:
## 크롤링에 오류는 확인되었으나, nv_type은 활용하도록 한다
all_df = pd.concat([place_df,naver_craw[['nv_name','nv_type']]],axis=1)

naver_craw.iloc[list(set(chk_index1) & set(chk_index2))] = np.NaN

all_df = pd.concat([all_df,naver_craw[['nv_address', 'nv_review_num', 'nv_blog_num',
       'nv_review', 'nv_dl_mood', 'nv_dl_topic', 'nv_dl_purpose', 'nv_dl_20']]],axis=1)
all_df

,CL_NM,AREA_NM_PRE,name_PRE,ADDR_PRE,LTNO_ADDR,GPS_LNGTD,GPS_LTTD,tags,like,heart,...,nv_name,nv_type,nv_address,nv_review_num,nv_blog_num,nv_review,nv_dl_mood,nv_dl_topic,nv_dl_purpose,nv_dl_20
0,테마관광지,감귤박물관,감귤박물관,제주특별자치도서귀포시효돈순환로441,제주특별자치도서귀포시신효동1200-6,126.607428496419,33.271593041773,#실내 #미술/박물관 #아이 #비.눈 #겨울 #실내관광지 #문화관광 #박물관 #어트...,5.0,190,...,서귀포감귤박물관,박물관,제주특별자치도서귀포시효돈순환로441,2180,883.0,0,NaN,NaN,NaN,NaN
1,공연전시,건강과성박물관,건강과성박물관,제주특별자치도서귀포시안덕면일주서로1611,제주특별자치도서귀포시안덕면감산리1736,126.346583683696,33.2548469661363,#실내 #미술/박물관 #커플 #비.눈 #사계절 #실내관광지 #문화관광 #박물관 #어...,6.0,110,...,건강과성박물관,박물관,제주특별자치도서귀포시안덕면일주서로1611,4415,543.0,0,NaN,NaN,NaN,NaN
2,공연전시,고흐의정원,고흐의정원,제주특별자치도서귀포시성산읍삼달신풍로126-5,제주특별자치도서귀포시성산읍삼달리1185-2,126.844086486445,33.3734094632983,#부모 #커플 #혼자 #친구 #아이 #사계절 #미술 #체험전시 #자연경관 #실내관광...,0.0,53,...,고흐의정원,"관람,체험",제주특별자치도서귀포시성산읍삼달신풍로126-5,1361,370.0,0,NaN,NaN,NaN,NaN
3,레저관광,김녕요트투어,김녕요트투어,제주특별자치도제주시구좌읍구좌해안로229-16,제주특별자치도제주시구좌읍김녕리4212-1,126.7372597689,33.5582479162063,#액티비티 #커플 #여름 #체험 #레저/체험 #수상레저 #어트랙션 #체험 #레저/체...,3.0,144,...,김녕요트투어,요트,제주특별자치도제주시구좌읍구좌해안로229-16,2011,489.0,0,NaN,NaN,NaN,NaN
4,기타,꽃썸,꽃썸,제주특별자치도서귀포시안덕면동광로151,제주특별자치도서귀포시안덕면동광리950-3,126.34015745353,33.3097350065685,#해변 #카페 #포토스팟 #휴식 #부모 #친구 #커플 #혼자 #안개/흐림 #맑음 #...,0.0,50,...,꽃썸카페,"카페,디저트",제주특별자치도서귀포시안덕면동광로151,27,13.0,0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1063,NaN,후포해변,후포해변,제주특별자치도제주시추자면대서5길87,제주특별자치도제주시추자면대서리267,126.28954186844,33.9623416994265,#해변 #경관/포토 #자연경관 #포토스팟 #포토스팟,0.0,4,...,후포해변,"해수욕장,해변",제주특별자치도제주시추자면,6,8.0,NaN,NaN,NaN,NaN,NaN
1064,NaN,훈데르트바서파크,훈데르트바서파크,제주특별자치도제주시우도면우도해안길32-12,제주특별자치도제주시우도면연평리1706-1,126.955738014169,33.4931582693186,#문화관광 #건축문화기행 #테마공원 #포토스팟 #포토스팟,0.0,9,...,훈데르트바서파크,테마파크,제주특별자치도제주시우도면우도해안길32-12,347,202.0,"{'볼거리가 많아요': 245, '사진이 잘 나와요': 169, '주차하기 편해요'...",NaN,NaN,NaN,NaN
1065,NaN,휘닉스르쏠레이테라피,휘닉스르쏠레이테라피,제주특별자치도서귀포시성산읍섭지코지로107,제주특별자치도서귀포시성산읍고성리127-2,126.926442688389,33.4260423785235,#스파 #체험 #호텔 #어트랙션 #어트랙션,7.0,5,...,휘닉스르쏠레이테라피,no_result,no_result,no_result,0.0,0,no_result,no_result,no_result,no_result
1066,NaN,휴림,휴림,제주특별자치도제주시애월읍광령남서길40,제주특별자치도제주시애월읍광령리2873,126.428787959221,33.4437798639377,#부모 #친구 #아이 #커플 #액티비티 #야영장 #캠핑장 #체험 #레저/체험 #어린...,0.0,8,...,휴림,"캠핑,야영장",제주특별자치도제주시애월읍광령남서길40,136,113.0,"{'그늘이 많아요': 33, '조용히 쉬기 좋아요': 29, '주차하기 편해요': ...",NaN,NaN,NaN,NaN


### 4-(2) 네이버 크롤링 데이터로 기반 분류값 보완

In [57]:
# 네이버지도에서 분류가 없는 경우는 이미 nan으로 저장되어 있었다..
all_df[all_df.nv_type=="no_result"].CL_NM.drop_duplicates()

51    NaN
Name: CL_NM, dtype: object

In [58]:
#분류값은 9가지...
all_df.CL_NM.unique()

array(['테마관광지', '공연전시', '레저관광', '기타', '레저/체험', '테마파크', '자연', '문화관광', '키즈',
       nan], dtype=object)

In [59]:
#분류값을 정리해서 [대분류(L_cat)]을 만들자
all_df['CAT_PRE'] = np.NaN

for i in range(0, len(all_df)):
    
    if all_df.CL_NM[i] == "자연":
            all_df.CAT_PRE[i] = "자연"
            
    elif all_df.nv_type[i] in ["기념물", "문화재", "봉우리,고지", "식물원,수목원", "폭포", "해수욕장,해변", "휴양림,산림욕장","자연공원"]:
            all_df.CAT_PRE[i] = "자연"
        
    elif all_df.CL_NM[i] in ["레저관광", "레저/체험"] :
            all_df.CAT_PRE[i] = "레저/체험"
            
    elif all_df.CL_NM[i] in ["테마관광지", "공연전시", "테마파크","문화관광","키즈"] :
            all_df.CAT_PRE[i] = "테마파크/공연/전시"

In [60]:
all_df.CAT_PRE.isna().sum() #기타,미분류 925개 

925

In [61]:
cat = all_df[['CAT_PRE','nv_type','AREA_NM_PRE']].groupby(['nv_type','CAT_PRE']).count()
cat = cat.reset_index(drop=False) 
cat

,nv_type,CAT_PRE,AREA_NM_PRE
0,공연장,테마파크/공연/전시,1
1,"관람,체험",테마파크/공연/전시,4
2,기념물,자연,10
3,동물원,테마파크/공연/전시,1
4,레일바이크,레저/체험,1
5,"레저,테마",레저/체험,1
6,레포츠시설,레저/체험,1
7,문화재,자연,27
8,미술관,테마파크/공연/전시,1
9,박물관,테마파크/공연/전시,7


In [62]:
#분류되는 경향(cat)에 따라 분류값 넣기
for j in range(0, len(cat)) :
    nv_cat = cat.nv_type.iloc[j]
    pre_cat = cat.CAT_PRE.iloc[j]
    all_df.CAT_PRE[(all_df.CAT_PRE.isna()) * (all_df.nv_type == nv_cat)] = pre_cat

In [63]:
cat_na_index = all_df[all_df.CAT_PRE.isna()].index
all_df.iloc[cat_na_index] #기타,미분류가 -> 763 

,CL_NM,AREA_NM_PRE,name_PRE,ADDR_PRE,LTNO_ADDR,GPS_LNGTD,GPS_LTTD,tags,like,heart,...,nv_type,nv_address,nv_review_num,nv_blog_num,nv_review,nv_dl_mood,nv_dl_topic,nv_dl_purpose,nv_dl_20,CAT_PRE
4,기타,꽃썸,꽃썸,제주특별자치도서귀포시안덕면동광로151,제주특별자치도서귀포시안덕면동광리950-3,126.34015745353,33.3097350065685,#해변 #카페 #포토스팟 #휴식 #부모 #친구 #커플 #혼자 #안개/흐림 #맑음 #...,0.0,50,...,"카페,디저트",제주특별자치도서귀포시안덕면동광로151,27,13.0,0,NaN,NaN,NaN,NaN,NaN
51,NaN,1112도로,1112도로,제주특별자치도제주시명림로584,제주특별자치도제주시봉개동산78-1,126.625203401207,33.4400488067904,#산 #드라이브 #흐림 #자연경관 #자연경관,4.0,125,...,no_result,no_result,no_result,0.0,0,no_result,no_result,no_result,no_result,NaN
52,NaN,43해원방사탑,43해원방사탑,제주특별자치도제주시신산로82,제주특별자치도제주시일도이동837-20,126.534739597022,33.5054858498659,#4.3 #문화유적지 #문화관광 #제주4.3 #역사유적 #어트랙션 #제주4.3 #역...,0.0,1,...,no_result,no_result,no_result,0.0,0,no_result,no_result,no_result,no_result,NaN
53,NaN,516도로숲터널,516도로숲터널,제주특별자치도서귀포시남원읍신례리산2-1,제주특별자치도서귀포시남원읍신례리산2-1,126.605070169993,33.3641266493346,#드라이브 #맑음 #자연경관 #숲 #단풍 #제주명품42 #포토스팟 #제주7대도로,3.0,190,...,터널,제주특별자치도서귀포시남원읍,0,265.0,NaN,NaN,NaN,NaN,NaN,NaN
55,NaN,9연대본부옛터,9연대본부옛터,제주특별자치도서귀포시대정읍일주서로2497,NaN,NaN,NaN,#4.3 #문화유적지 #문화관광 #제주4.3 #역사유적 #어트랙션 #제주4.3 #역...,1.0,0,...,no_result,no_result,no_result,0.0,0,no_result,no_result,no_result,no_result,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1059,NaN,황우럭만화천국사회적협동조합,황우럭만화천국사회적협동조합,제주특별자치도제주시한림읍내동남길7-3,제주특별자치도제주시한림읍한림리1055,126.266560879597,33.414260650609,#친구 #아이 #체험관광 #어린이 #어트랙션 #어린이 #어트랙션,0.0,0,...,no_result,no_result,no_result,0.0,0,no_result,no_result,no_result,no_result,NaN
1061,NaN,황우지해안열두굴,황우지해안열두굴,제주특별자치도서귀포시서홍동764-1,제주특별자치도서귀포시서홍동764-1,126.550718986967,33.2420017273439,#문화유적지 #해변 #커플 #친구 #맑음 #자연경관 #역사유적 #친구 #맑음 #자연...,2.0,73,...,동굴,제주특별자치도서귀포시서홍동,0,260.0,NaN,NaN,NaN,NaN,NaN,NaN
1062,NaN,효명사,효명사,제주특별자치도서귀포시남원읍516로815-41,제주특별자치도서귀포시남원읍하례리산45,126.594171841829,33.3232979713049,#경관/포토 #휴식/힐링 #절 #사찰 #포토스팟 #어트랙션 #사찰 #포토스팟 #어트랙션,2.0,12,...,불교,제주특별자치도서귀포시남원읍516로815-41,0,245.0,NaN,NaN,NaN,NaN,NaN,NaN
1065,NaN,휘닉스르쏠레이테라피,휘닉스르쏠레이테라피,제주특별자치도서귀포시성산읍섭지코지로107,제주특별자치도서귀포시성산읍고성리127-2,126.926442688389,33.4260423785235,#스파 #체험 #호텔 #어트랙션 #어트랙션,7.0,5,...,no_result,no_result,no_result,0.0,0,no_result,no_result,no_result,no_result,NaN


In [64]:
all_df.nv_type.iloc[cat_na_index].unique()

array(['카페,디저트', 'no_result', '터널', '오름', '협회,단체', '섬', '공방', '베이커리',
       '개신교', '마을회관', '안경원', '갤러리카페', '자연,생태공원', '체험여행', '펜션', '산,고개',
       '페리,해운', '천연화장품', '거리,골목', '숲,숲길', '복합문화공간', '해물,생선요리', '캠핑,야영장',
       '보물', '불교', '템플스테이', '한식', '선착장', '스킨스쿠버', '배낚시', '서핑,윈드서핑', '항구',
       '주말농장', '문화거리', '동굴', '전통,민속주점', '기념관,생가', '갤러리,화랑', '도서관',
       '어린이도서관', '바위', '체험마을', '호수,연못,저수지', '도시,테마공원', '승마장', '자전거',
       '향토음식', '행정지명', '올레둘레길', '전통식품', '촬영장소', '문화,유적', '초등학교', '유원지',
       '도보코스', '슈퍼,마트', '방면정보', '골프연습장', '가죽제품제작', '서바이벌게임', '경마장-일반',
       '공사,공단', '피부,체형관리', '공원', '등대', '특산물,관광민예품', '천주교', '부속시설', '도로명칭',
       '가공식품', '관광안내소', '강,하천', '관광농원,팜스테이', '생태보호구역', '5성급', '근린공원',
       '발관리', '계곡', '보관,저장', '요가원', '전망대', '성곽명', '곡물재배', '퍼블릭골프장', '과일',
       '키즈카페,실내놀이터', '골짜기,고개', '낚시', '푸드코트', '드라이브', '일식당', '일반여객터미널',
       '올레길', '차,커피', '궁도장', '문화,예술회관', '종합병원', '천문대', '해안명', '먹자거리',
       '보건진료소', '전동모빌리티', '화훼재배', '들,평야', '농공시설', '북카페', '한방병원', '저수지,제',

In [65]:
#직접 분류
for i in cat_na_index :
    
    if all_df.nv_type[i] in ["오름", "호수,연못,저수지","섬","안경원","자연,생태공원","산,고개","페리,해운","숲,숲길","해물,생선요리","선착장","항구","동굴","바위","들,평야"
                             "호수,연못,저수지","올레둘레길","도보코스","슈퍼,마트","방면정보","관광안내소","강,하천","생태보호구역","근린공원","계곡","전망대","골짜기,고개","푸드코트","일식당","일반여객터미널",
                            "올레길","해안명","성산오조리해안","저수지,제","수영장","도립공원","민박","숲","산","국립공원","코스명","산책로","들,평야"]:
            all_df.CAT_PRE[i] = "자연"
        
    elif all_df.nv_type[i] in ["마을회관", "협회,단체","공방","천연화장품","체험여행","캠핑,야영장","템플스테이","스킨스쿠버","배낚시","서핑,윈드서핑","주말농장","체험마을",
                               "승마장","자전거","전통식품","골프연습장","가죽제품제작","서바이벌게임","경마장-일반","롯데호텔브이스파","특산물,관광민예품","가공식품",
                              "관광농원,팜스테이","발관리","요가원","퍼블릭골프장","과일","낚시","궁도장","농공시설","도보여행","과실재배","카트장","떡,한과","콘도,리조트","회관","종합도소매","생선회",
                              "볼링장","임대,대여","짚라인","클럽하우스","회원제골프장","오토바이","행글라이딩,패러글라이딩","요가,명상","골프장코스","IT서비스","전문디자인","오락,레저용품","비행,항공레저","기업","테마카페"] :
            all_df.CAT_PRE[i] = "레저/체험"
            
    elif all_df.nv_type[i] in ["터널", "갤러리카페", "개신교","거리,골목","복합문화공간","보물","불교","문화거리","기념관,생가","갤러리,화랑","도서관","어린이도서관","양어,양식업",
                               "도시,테마공원","촬영장소","문화,유적","초등학교","유원지","공사,공단","공원","등대","천주교","부속시설","5성급","보관,저장","성곽명","곡물재배",
                              "키즈카페,실내놀이터","드라이브","차,커피","문화,예술회관","천문대","화훼재배","북카페","편의점","워터파크","아쿠아리움","라면","돼지고기구이","문화시설","화장품,향수","카페",
                              "컨벤션센터","4성급","문,창문","공연,연극시설","3성급","도로시설"] :
            all_df.CAT_PRE[i] = "테마파크/공연/전시"


In [66]:
#기타,미분류 개수
all_df.CAT_PRE.isna().sum()

242

In [67]:
for i in cat_na_index :
    
    if all_df.name_PRE[i] in ["구좌해안로","노을해안로","녹산로유채꽃길","도두항","들렁모루산책길","사계용머리해안"]:
            all_df.CAT_PRE[i] = "자연"
        
    elif all_df.name_PRE[i] in ["감귤상웨빵체험","목장카페드르쿰다","무릉도원마을","제주조천스위스마을","중문카라반","한라산아래첫마을","산방산용머리해안지질트레일"] :
            all_df.CAT_PRE[i] = "레저/체험"
            
    elif all_df.name_PRE[i] in ["제주인디"] :
            all_df.CAT_PRE[i] = "테마파크/공연/전시"

all_df.CAT_PRE.isna().sum()

229

In [68]:
all_df.isna().sum() #네이버 크롤링 데이터에서 채워지지 않은 row가 여럿 있다

CL_NM              1015
AREA_NM_PRE           0
name_PRE              0
ADDR_PRE              0
LTNO_ADDR            44
GPS_LNGTD            44
GPS_LTTD             44
tags                  0
like                  0
heart                 0
detail_text         102
detail_side_box     101
MAR_VIEW_CO           0
APR_VIEW_CO           0
MAY_VIEW_CO           0
SEP_VIEW_CO           0
OCT_VIEW_CO           0
NOV_VIEW_CO           0
MAR_VISIT_PRICE    1031
APR_VISIT_PRICE    1029
MAY_VISIT_PRICE    1029
SEP_VISIT_PRICE    1038
OCT_VISIT_PRICE    1037
NOV_VISIT_PRICE    1023
MAR_VISIT_NUM      1031
APR_VISIT_NUM      1029
MAY_VISIT_ NUM     1029
SEP_VISIT_NUM      1038
OCT_VISIT_NUM      1037
NOV_VISIT_NUM      1023
nv_name               0
nv_type               0
nv_address           24
nv_review_num        24
nv_blog_num          24
nv_review           525
nv_dl_mood          870
nv_dl_topic         868
nv_dl_purpose       869
nv_dl_20            868
CAT_PRE             229
dtype: int64

### 4-(2) 네이버 크롤링 데이터 보완

누락 데이터 재 크롤링

In [69]:
place_craw = pd.read_csv('data/관광지데이터_네이버크롤링_835.csv', encoding='utf-8-sig')

place_craw = place_craw.drop_duplicates(['org_list','name_list'])
place_craw['name_list'] = list(map(lambda x : name_preprocessing(x), place_craw['name_list']))

place_craw['address_list'] = place_craw['address_list'].str.replace(pat="제주 ",repl="제주특별자치도 ",regex=False)
place_craw['address_list'] = place_craw['address_list'].str.replace(pat="제주특별자치 ",repl="제주특별자치도 ",regex=False)

In [70]:
# 2차 크롤링 데이터로 보완하기
all_df = all_df.drop(['nv_review_num', 'nv_blog_num', 'nv_review', 'nv_dl_mood','nv_dl_topic', 'nv_dl_purpose', 'nv_dl_20'], axis=1)
place = pd.merge(left = all_df, right = place_craw, how="left", left_on=["name_PRE"], right_on=["org_list"])
place = place.dropna(subset = ['name_list', 'address_list']).reset_index(drop=True) #검색 안된애는 삭제

In [71]:
# 크롤링 데이터의 적절성 확인1 : 주소가 일치하는가
addr_pre =place.address_list.str[0:15].str.replace(" ","")
chk_index1 = place[(place.ADDR_PRE.str[0:13] != addr_pre) * (place.LTNO_ADDR.str[0:13] != addr_pre)].index

# 크롤링 데이터의 적절성 확인2 : 이름이 일치하는가
chk_index2 = []
for i in range(0,len(place)):
    chk_name = place.name_list.iloc[i]
    
    if (place.AREA_NM_PRE.iloc[i] not in chk_name) * (place.name_PRE.iloc[i] not in chk_name)  * (chk_name not in place.name_PRE.iloc[i])  != 0 : #어디도 포함되지 않으면
        chk_index2.append(i)    
        
chk_index_inter = list(set(chk_index1) & set(chk_index2))
place[['name_PRE','name_list','LTNO_ADDR','address_list']].iloc[chk_index_inter]

,name_PRE,name_list,LTNO_ADDR,address_list
640,제주올레18코스,제주올레길18코스,제주특별자치도제주시삼양일동1131-2,제주특별자치도 제주시 삼양이동
642,제주올레1코스,제주올레길1-1코스,제주특별자치도서귀포시성산읍오조리396-1,제주특별자치도 제주시 우도면 연평리 2473-1
646,제주올레3코스,제주올레길3코스,제주특별자치도서귀포시표선면하천리87-1,제주특별자치도 서귀포시 성산읍 신풍리 17-1
647,제주올레4코스,제주올레길4코스,제주특별자치도서귀포시남원읍태흥리200-2,제주특별자치도 서귀포시 표선면 토산중앙로49번길 24-117
650,제주올레7-1코스,제주올레길7-1코스,제주특별자치도서귀포시서호동553-1,제주특별자치도 서귀포시 호근로 110
651,제주올레7코스,제주올레길7코스,제주특별자치도서귀포시서홍동795-1,제주특별자치도 서귀포시 강정동
652,제주올레8코스,제주올레길8코스,제주특별자치도서귀포시하예동532-3,제주특별자치도 서귀포시 색달동 2091-5
657,제주인디,동광분교인디이스트,제주특별자치도서귀포시성산읍삼달리1010,제주특별자치도 서귀포시 안덕면 동광로 107 서광초등학교동광분교장
786,한라산어리목코스,한라산국립공원어리목탐방로,제주특별자치도제주시해안동산220-13,제주특별자치도 제주시 애월읍 광령리
667,제주컨트리클럽,캐슬렉스제주cc,제주특별자치도제주시영평동2263-5,제주특별자치도 서귀포시 안덕면 평화로 1241


In [72]:
del_list = ["제주인디", "함덕카페거리", "누운오름","제주힐링명상센터", "가메오름","이호테우말등대","사려니오름","추사유배길-2코스연인의길",
"제주다이브체험다이빙","로로요가","제주스쿠터여행","서귀포궁도장","제주올레14코드","표선세화해안도로","제주올레1코스"]

del_index = []
for i in chk_index_inter :
    if place.name_PRE.iloc[i] in del_list :
        del_index.append(i)

del_index.append(place[place.name_PRE == "삼화포구"].index[0])

place = place.drop(del_index, axis=0).reset_index(drop=True)
place

,CL_NM,AREA_NM_PRE,name_PRE,ADDR_PRE,LTNO_ADDR,GPS_LNGTD,GPS_LTTD,tags,like,heart,...,type_list,address_list,visitor_review_num_list,blog_review_num_list,visitor_review_list,dl_mood_list,dl_topic_list,dl_why_list,dl_20_pop_list,org_list
0,테마관광지,감귤박물관,감귤박물관,제주특별자치도서귀포시효돈순환로441,제주특별자치도서귀포시신효동1200-6,126.607428496419,33.271593041773,#실내 #미술/박물관 #아이 #비.눈 #겨울 #실내관광지 #문화관광 #박물관 #어트...,5.0,190,...,박물관,제주특별자치도 서귀포시 효돈순환로 441,2180.0,886.0,"{'가격이 합리적이에요': 153, '주차하기 편해요': 108, '유익해요': 9...","화려한, 조그마한, 재미있는, 이국적","한라봉, 폭포, 박물관, 식물원, 박람회","승마체험, 가족여행, 힐링, 유물",70.25449999999998%,감귤박물관
1,공연전시,건강과성박물관,건강과성박물관,제주특별자치도서귀포시안덕면일주서로1611,제주특별자치도서귀포시안덕면감산리1736,126.346583683696,33.2548469661363,#실내 #미술/박물관 #커플 #비.눈 #사계절 #실내관광지 #문화관광 #박물관 #어...,6.0,110,...,박물관,제주특별자치도 서귀포시 안덕면 일주서로 1611 건강과 성 박물관,4422.0,542.0,"{'주차하기 편해요': 352, '유익해요': 342, '설명이 잘 되어있어요': ...","재미있는, 이국적, 편안한, 운치있는","온천, 해변, 박물관, 성인박물관, 흑돼지","웰빙, 경치, 비오는날, 드라이브, 체험관",38.55171428571426%,건강과성박물관
2,공연전시,고흐의정원,고흐의정원,제주특별자치도서귀포시성산읍삼달신풍로126-5,제주특별자치도서귀포시성산읍삼달리1185-2,126.844086486445,33.3734094632983,#부모 #커플 #혼자 #친구 #아이 #사계절 #미술 #체험전시 #자연경관 #실내관광...,0.0,53,...,"관람,체험",제주특별자치도 서귀포시 성산읍 삼달신풍로 126-5,1421.0,373.0,0,NaN,NaN,NaN,NaN,고흐의정원
3,레저관광,김녕요트투어,김녕요트투어,제주특별자치도제주시구좌읍구좌해안로229-16,제주특별자치도제주시구좌읍김녕리4212-1,126.7372597689,33.5582479162063,#액티비티 #커플 #여름 #체험 #레저/체험 #수상레저 #어트랙션 #체험 #레저/체...,3.0,144,...,요트,제주특별자치도 제주시 구좌읍 구좌해안로 229-16,2030.0,490.0,"{'친절해요': 366, '뷰가 좋아요': 354, '안전하게 관리해요': 213,...",NaN,NaN,NaN,NaN,김녕요트투어
4,공연전시,노형수퍼마켙,노형수퍼마켙,제주특별자치도제주시노형로89,제주특별자치도제주시해안동2137-1,126.455168486651,33.4649423421818,#부모 #커플 #혼자 #친구 #아이 #휴식/힐링 #테마공원 #미술/박물관 #사계절 ...,3.0,83,...,미술관,제주특별자치도 제주시 노형로 89,4976.0,2161.0,0,NaN,NaN,NaN,NaN,노형수퍼마켙
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
811,NaN,효명사,효명사,제주특별자치도서귀포시남원읍516로815-41,제주특별자치도서귀포시남원읍하례리산45,126.594171841829,33.3232979713049,#경관/포토 #휴식/힐링 #절 #사찰 #포토스팟 #어트랙션 #사찰 #포토스팟 #어트랙션,2.0,12,...,불교,제주특별자치도 서귀포시 남원읍 516로 815-41,0.0,246.0,NaN,NaN,NaN,NaN,NaN,효명사
812,NaN,후포해변,후포해변,제주특별자치도제주시추자면대서5길87,제주특별자치도제주시추자면대서리267,126.28954186844,33.9623416994265,#해변 #경관/포토 #자연경관 #포토스팟 #포토스팟,0.0,4,...,"해수욕장,해변",제주특별자치도 제주시 추자면 대서리,6.0,9.0,NaN,NaN,NaN,NaN,NaN,후포해변
813,NaN,훈데르트바서파크,훈데르트바서파크,제주특별자치도제주시우도면우도해안길32-12,제주특별자치도제주시우도면연평리1706-1,126.955738014169,33.4931582693186,#문화관광 #건축문화기행 #테마공원 #포토스팟 #포토스팟,0.0,9,...,테마파크,제주특별자치도 제주시 우도면 우도해안길 32-12,374.0,213.0,"{'볼거리가 많아요': 254, '사진이 잘 나와요': 180, '주차하기 편해요'...",NaN,NaN,NaN,NaN,훈데르트바서파크
814,NaN,휴림,휴림,제주특별자치도제주시애월읍광령남서길40,제주특별자치도제주시애월읍광령리2873,126.428787959221,33.4437798639377,#부모 #친구 #아이 #커플 #액티비티 #야영장 #캠핑장 #체험 #레저/체험 #어린...,0.0,8,...,"캠핑,야영장",제주특별자치도 제주시 애월읍 광령남서길 40,136.0,113.0,"{'그늘이 많아요': 33, '조용히 쉬기 좋아요': 29, '주차하기 편해요': ...",NaN,NaN,NaN,NaN,휴림


In [73]:
place = place.dropna(subset = ['GPS_LNGTD', 'address_list', 'detail_text']).reset_index(drop=True) #검색 안된애는 삭제

In [74]:
#place.to_csv('PRE_4(2)_관광지데이터1011.csv', encoding='utf-8-sig', index=False)
place_df = place

## 5. 탄소배출량 데이터 병합

In [75]:
#place_df = pd.read_csv('PRE_4(2)_관광지데이터1011.csv',encoding='utf-8-sig')
emit = pd.read_csv('data/PRE_CO2_지번단위탄소배출량_meanmax.csv',encoding='utf-8-sig')
emit = emit[['LTNO_ADDR', 'co2_max']]

In [76]:
#주소 정리
emit['LTNO_ADDR']=emit['LTNO_ADDR'].str.replace(pat='번지',repl='',regex=False)
emit['LTNO_ADDR'] = list(map(lambda x : addr_preprocessing3(x), emit['LTNO_ADDR'])) ; emit

,LTNO_ADDR,co2_max
0,제주특별자치도서귀포시강정동0,0.009123
1,제주특별자치도서귀포시강정동1026-1,0.109615
2,제주특별자치도서귀포시강정동1045-5,0.000990
3,제주특별자치도서귀포시강정동108-10,0.054986
4,제주특별자치도서귀포시강정동1155,0.002114
...,...,...
34515,제주특별자치도제주시회천동656-2,0.080382
34516,제주특별자치도제주시회천동722-2,0.005129
34517,제주특별자치도제주시회천동742-2,0.002098
34518,제주특별자치도제주시회천동79-2,0.031270


In [77]:
all_df = pd.merge(left = place_df, right = emit, how = 'left', on = 'LTNO_ADDR').reset_index(drop=True)
all_df

,CL_NM,AREA_NM_PRE,name_PRE,ADDR_PRE,LTNO_ADDR,GPS_LNGTD,GPS_LTTD,tags,like,heart,...,address_list,visitor_review_num_list,blog_review_num_list,visitor_review_list,dl_mood_list,dl_topic_list,dl_why_list,dl_20_pop_list,org_list,co2_max
0,테마관광지,감귤박물관,감귤박물관,제주특별자치도서귀포시효돈순환로441,제주특별자치도서귀포시신효동1200-6,126.607428496419,33.271593041773,#실내 #미술/박물관 #아이 #비.눈 #겨울 #실내관광지 #문화관광 #박물관 #어트...,5.0,190,...,제주특별자치도 서귀포시 효돈순환로 441,2180.0,886.0,"{'가격이 합리적이에요': 153, '주차하기 편해요': 108, '유익해요': 9...","화려한, 조그마한, 재미있는, 이국적","한라봉, 폭포, 박물관, 식물원, 박람회","승마체험, 가족여행, 힐링, 유물",70.25449999999998%,감귤박물관,0.023639
1,공연전시,건강과성박물관,건강과성박물관,제주특별자치도서귀포시안덕면일주서로1611,제주특별자치도서귀포시안덕면감산리1736,126.346583683696,33.2548469661363,#실내 #미술/박물관 #커플 #비.눈 #사계절 #실내관광지 #문화관광 #박물관 #어...,6.0,110,...,제주특별자치도 서귀포시 안덕면 일주서로 1611 건강과 성 박물관,4422.0,542.0,"{'주차하기 편해요': 352, '유익해요': 342, '설명이 잘 되어있어요': ...","재미있는, 이국적, 편안한, 운치있는","온천, 해변, 박물관, 성인박물관, 흑돼지","웰빙, 경치, 비오는날, 드라이브, 체험관",38.55171428571426%,건강과성박물관,0.507125
2,공연전시,고흐의정원,고흐의정원,제주특별자치도서귀포시성산읍삼달신풍로126-5,제주특별자치도서귀포시성산읍삼달리1185-2,126.844086486445,33.3734094632983,#부모 #커플 #혼자 #친구 #아이 #사계절 #미술 #체험전시 #자연경관 #실내관광...,0.0,53,...,제주특별자치도 서귀포시 성산읍 삼달신풍로 126-5,1421.0,373.0,0,NaN,NaN,NaN,NaN,고흐의정원,0.021090
3,레저관광,김녕요트투어,김녕요트투어,제주특별자치도제주시구좌읍구좌해안로229-16,제주특별자치도제주시구좌읍김녕리4212-1,126.7372597689,33.5582479162063,#액티비티 #커플 #여름 #체험 #레저/체험 #수상레저 #어트랙션 #체험 #레저/체...,3.0,144,...,제주특별자치도 제주시 구좌읍 구좌해안로 229-16,2030.0,490.0,"{'친절해요': 366, '뷰가 좋아요': 354, '안전하게 관리해요': 213,...",NaN,NaN,NaN,NaN,김녕요트투어,0.282392
4,공연전시,노형수퍼마켙,노형수퍼마켙,제주특별자치도제주시노형로89,제주특별자치도제주시해안동2137-1,126.455168486651,33.4649423421818,#부모 #커플 #혼자 #친구 #아이 #휴식/힐링 #테마공원 #미술/박물관 #사계절 ...,3.0,83,...,제주특별자치도 제주시 노형로 89,4976.0,2161.0,0,NaN,NaN,NaN,NaN,노형수퍼마켙,0.403384
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
715,NaN,효명사,효명사,제주특별자치도서귀포시남원읍516로815-41,제주특별자치도서귀포시남원읍하례리산45,126.594171841829,33.3232979713049,#경관/포토 #휴식/힐링 #절 #사찰 #포토스팟 #어트랙션 #사찰 #포토스팟 #어트랙션,2.0,12,...,제주특별자치도 서귀포시 남원읍 516로 815-41,0.0,246.0,NaN,NaN,NaN,NaN,NaN,효명사,NaN
716,NaN,후포해변,후포해변,제주특별자치도제주시추자면대서5길87,제주특별자치도제주시추자면대서리267,126.28954186844,33.9623416994265,#해변 #경관/포토 #자연경관 #포토스팟 #포토스팟,0.0,4,...,제주특별자치도 제주시 추자면 대서리,6.0,9.0,NaN,NaN,NaN,NaN,NaN,후포해변,0.014183
717,NaN,훈데르트바서파크,훈데르트바서파크,제주특별자치도제주시우도면우도해안길32-12,제주특별자치도제주시우도면연평리1706-1,126.955738014169,33.4931582693186,#문화관광 #건축문화기행 #테마공원 #포토스팟 #포토스팟,0.0,9,...,제주특별자치도 제주시 우도면 우도해안길 32-12,374.0,213.0,"{'볼거리가 많아요': 254, '사진이 잘 나와요': 180, '주차하기 편해요'...",NaN,NaN,NaN,NaN,훈데르트바서파크,NaN
718,NaN,휴림,휴림,제주특별자치도제주시애월읍광령남서길40,제주특별자치도제주시애월읍광령리2873,126.428787959221,33.4437798639377,#부모 #친구 #아이 #커플 #액티비티 #야영장 #캠핑장 #체험 #레저/체험 #어린...,0.0,8,...,제주특별자치도 제주시 애월읍 광령남서길 40,136.0,113.0,"{'그늘이 많아요': 33, '조용히 쉬기 좋아요': 29, '주차하기 편해요': ...",NaN,NaN,NaN,NaN,휴림,0.002456


## 6. 변수 정리 

- 비짓제주 월별 조회수 모두 합하기
- 입장객수 최대값으로 선택
- 인원수로 나누어 인당 가격정보 계산
- 가격정보 최대값으로 선택


In [78]:
#조회수는 총합으로 정리
all_df['VJ_VIEW'] = all_df[['MAR_VIEW_CO','APR_VIEW_CO', 'MAY_VIEW_CO', 'SEP_VIEW_CO', 'OCT_VIEW_CO','NOV_VIEW_CO']].sum(axis=1)

#입장 최대값으로 정리
all_df['NUM_MAX'] = all_df[['MAR_VISIT_NUM', 'APR_VISIT_NUM', 'MAY_VISIT_ NUM', 'SEP_VISIT_NUM','OCT_VISIT_NUM', 'NOV_VISIT_NUM']].max(axis=1)

#인당 가격 = price/num
for i in range(0,6):
    all_df.iloc[:,18+i] = (all_df.iloc[:,18+i]/all_df.iloc[:,24+i]).round(0)

#가격 최대값으로 정리
all_df['PREICE_MAX'] = all_df[['MAR_VISIT_PRICE', 'APR_VISIT_PRICE', 'MAY_VISIT_PRICE','SEP_VISIT_PRICE', 'OCT_VISIT_PRICE', 'NOV_VISIT_PRICE']].max(axis=1)

In [79]:
# 안 쓸 열 drop
all_df = all_df.drop(['ADDR_PRE','MAR_VISIT_PRICE','APR_VISIT_PRICE', 'MAY_VISIT_PRICE', 'SEP_VISIT_PRICE','OCT_VISIT_PRICE', 'NOV_VISIT_PRICE',],axis=1) #월별 자료는 drop하고 max 자료만 받기

## 7. 업종별 평균으로 탄소배출량 결측치 대체

In [80]:
cat_pre_mean = all_df[['CAT_PRE','co2_max','NUM_MAX','PREICE_MAX']].groupby(['CAT_PRE']).mean() ;cat_pre_mean
cat_pre_mean = cat_pre_mean.reset_index(drop=False)
cat_pre_mean

,CAT_PRE,co2_max,NUM_MAX,PREICE_MAX
0,레저/체험,0.363131,533.666667,18602.666667
1,자연,0.307926,76346.153846,6094.230769
2,테마파크/공연/전시,0.592544,3636.423077,11851.961538


In [81]:
co2_na = all_df[all_df.co2_max.isna()].index

for i in co2_na:
    if (all_df.CAT_PRE.iloc[i] == cat_pre_mean.CAT_PRE[0]) :
        all_df.co2_max.iloc[i] = cat_pre_mean.co2_max[0]
        
    elif (all_df.CAT_PRE.iloc[i] == cat_pre_mean.CAT_PRE[1]) :
        all_df.co2_max.iloc[i] = cat_pre_mean.co2_max[1]
        
    elif (all_df.CAT_PRE.iloc[i] == cat_pre_mean.CAT_PRE[2]) :
        all_df.co2_max.iloc[i] = cat_pre_mean.co2_max[2]

In [82]:
num_na = all_df[all_df.NUM_MAX.isna()].index

for i in num_na:
    if (all_df.CAT_PRE.iloc[i] == cat_pre_mean.CAT_PRE[0]) :
        all_df.NUM_MAX.iloc[i] = cat_pre_mean.NUM_MAX[0]
        
    elif (all_df.CAT_PRE.iloc[i] == cat_pre_mean.CAT_PRE[1]) :
        all_df.NUM_MAX.iloc[i] = cat_pre_mean.NUM_MAX[1]
        
    elif (all_df.CAT_PRE.iloc[i] == cat_pre_mean.CAT_PRE[2]) :
        all_df.NUM_MAX.iloc[i] = cat_pre_mean.NUM_MAX[2]

In [83]:
#all_df.to_csv('PRE_real_final관광지데이터_1011.csv', encoding='utf-8-sig', index=False) 

## 8. 주소 오류 보완

In [84]:
vj_craw_spot = pd.read_csv('data/VISITJEJU_관광지정보크롤링.csv')

In [85]:
vj_craw_add = vj_craw_spot[['name','address']]
vj_craw_add['name'] = list(map(lambda x : name_preprocessing(x), vj_craw_add['name']))
vj_craw_add.columns = ['name_PRE','address']
vj_craw_add = vj_craw_add.drop_duplicates()

In [86]:
place_df2 = pd.merge(left = place_df, right = vj_craw_add, how = "left", on = "name_PRE")    
place_df2

,CL_NM,AREA_NM_PRE,name_PRE,ADDR_PRE,LTNO_ADDR,GPS_LNGTD,GPS_LTTD,tags,like,heart,...,address_list,visitor_review_num_list,blog_review_num_list,visitor_review_list,dl_mood_list,dl_topic_list,dl_why_list,dl_20_pop_list,org_list,address
0,테마관광지,감귤박물관,감귤박물관,제주특별자치도서귀포시효돈순환로441,제주특별자치도서귀포시신효동1200-6,126.607428496419,33.271593041773,#실내 #미술/박물관 #아이 #비.눈 #겨울 #실내관광지 #문화관광 #박물관 #어트...,5.0,190,...,제주특별자치도 서귀포시 효돈순환로 441,2180.0,886.0,"{'가격이 합리적이에요': 153, '주차하기 편해요': 108, '유익해요': 9...","화려한, 조그마한, 재미있는, 이국적","한라봉, 폭포, 박물관, 식물원, 박람회","승마체험, 가족여행, 힐링, 유물",70.25449999999998%,감귤박물관,제주특별자치도 서귀포시 효돈순환로 441
1,공연전시,건강과성박물관,건강과성박물관,제주특별자치도서귀포시안덕면일주서로1611,제주특별자치도서귀포시안덕면감산리1736,126.346583683696,33.2548469661363,#실내 #미술/박물관 #커플 #비.눈 #사계절 #실내관광지 #문화관광 #박물관 #어...,6.0,110,...,제주특별자치도 서귀포시 안덕면 일주서로 1611 건강과 성 박물관,4422.0,542.0,"{'주차하기 편해요': 352, '유익해요': 342, '설명이 잘 되어있어요': ...","재미있는, 이국적, 편안한, 운치있는","온천, 해변, 박물관, 성인박물관, 흑돼지","웰빙, 경치, 비오는날, 드라이브, 체험관",38.55171428571426%,건강과성박물관,제주특별자치도 서귀포시 안덕면 일주서로 1611
2,공연전시,고흐의정원,고흐의정원,제주특별자치도서귀포시성산읍삼달신풍로126-5,제주특별자치도서귀포시성산읍삼달리1185-2,126.844086486445,33.3734094632983,#부모 #커플 #혼자 #친구 #아이 #사계절 #미술 #체험전시 #자연경관 #실내관광...,0.0,53,...,제주특별자치도 서귀포시 성산읍 삼달신풍로 126-5,1421.0,373.0,0,NaN,NaN,NaN,NaN,고흐의정원,제주특별자치도 서귀포시 성산읍 삼달신풍로 126-5
3,레저관광,김녕요트투어,김녕요트투어,제주특별자치도제주시구좌읍구좌해안로229-16,제주특별자치도제주시구좌읍김녕리4212-1,126.7372597689,33.5582479162063,#액티비티 #커플 #여름 #체험 #레저/체험 #수상레저 #어트랙션 #체험 #레저/체...,3.0,144,...,제주특별자치도 제주시 구좌읍 구좌해안로 229-16,2030.0,490.0,"{'친절해요': 366, '뷰가 좋아요': 354, '안전하게 관리해요': 213,...",NaN,NaN,NaN,NaN,김녕요트투어,제주특별자치도 제주시 구좌읍 구좌해안로 229-16
4,공연전시,노형수퍼마켙,노형수퍼마켙,제주특별자치도제주시노형로89,제주특별자치도제주시해안동2137-1,126.455168486651,33.4649423421818,#부모 #커플 #혼자 #친구 #아이 #휴식/힐링 #테마공원 #미술/박물관 #사계절 ...,3.0,83,...,제주특별자치도 제주시 노형로 89,4976.0,2161.0,0,NaN,NaN,NaN,NaN,노형수퍼마켙,제주특별자치도 제주시 노형로 89
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
716,NaN,효명사,효명사,제주특별자치도서귀포시남원읍516로815-41,제주특별자치도서귀포시남원읍하례리산45,126.594171841829,33.3232979713049,#경관/포토 #휴식/힐링 #절 #사찰 #포토스팟 #어트랙션 #사찰 #포토스팟 #어트랙션,2.0,12,...,제주특별자치도 서귀포시 남원읍 516로 815-41,0.0,246.0,NaN,NaN,NaN,NaN,NaN,효명사,제주특별자치도 서귀포시 남원읍 516로 815-41
717,NaN,후포해변,후포해변,제주특별자치도제주시추자면대서5길87,제주특별자치도제주시추자면대서리267,126.28954186844,33.9623416994265,#해변 #경관/포토 #자연경관 #포토스팟 #포토스팟,0.0,4,...,제주특별자치도 제주시 추자면 대서리,6.0,9.0,NaN,NaN,NaN,NaN,NaN,후포해변,제주특별자치도 제주시 추자면 대서5길 87
718,NaN,훈데르트바서파크,훈데르트바서파크,제주특별자치도제주시우도면우도해안길32-12,제주특별자치도제주시우도면연평리1706-1,126.955738014169,33.4931582693186,#문화관광 #건축문화기행 #테마공원 #포토스팟 #포토스팟,0.0,9,...,제주특별자치도 제주시 우도면 우도해안길 32-12,374.0,213.0,"{'볼거리가 많아요': 254, '사진이 잘 나와요': 180, '주차하기 편해요'...",NaN,NaN,NaN,NaN,훈데르트바서파크,제주특별자치도 제주시 우도면 우도해안길 32-12
719,NaN,휴림,휴림,제주특별자치도제주시애월읍광령남서길40,제주특별자치도제주시애월읍광령리2873,126.428787959221,33.4437798639377,#부모 #친구 #아이 #커플 #액티비티 #야영장 #캠핑장 #체험 #레저/체험 #어린...,0.0,8,...,제주특별자치도 제주시 애월읍 광령남서길 40,136.0,113.0,"{'그늘이 많아요': 33, '조용히 쉬기 좋아요': 29, '주차하기 편해요': ...",NaN,NaN,NaN,NaN,휴림,제주특별자치도 제주시 애월읍 광령남서길 40


In [87]:
na_index = place_df2[place_df2.address.isna()].index
place_df2['address'].iloc[na_index] = place_df2['LTNO_ADDR'].iloc[na_index]
place_df2.address.isna().sum()

0

In [88]:
place_df2 = place_df2.drop(694, axis = 0)
place_df = place_df2.drop(['nv_name', 'nv_type', 'nv_address'], axis = 1)
place_df.shape

(720, 43)

In [89]:
place_df.shape

(720, 43)

In [90]:
place_df.to_csv('data/주소보완_관광지데이터.csv', encoding='utf-8-sig', index=False)